# Analysis Notebook for Comparing LLMs as Annotators for Skin Tone Perception

In [ ]:
# Packages 
import re
import pandas as pd
from pingouin import intraclass_corr
from typing import Union, Dict
import numpy as np
import pandas as pd
import krippendorff  
import statsmodels.formula.api as smf
from scipy.stats import chi2
import os
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

# R integration via rpy2
try:
    import rpy2.robjects as ro
    from rpy2.robjects import r
    from rpy2.robjects.packages import importr, isinstalled
    from rpy2.robjects.vectors import StrVector
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.conversion import localconverter

    # ensure CRAN mirror + required packages
    utils = importr('utils')
    try:
        utils.chooseCRANmirror(ind=1)
    except Exception:
        pass

    needed = [pkg for pkg in ("lme4", "emmeans") if not isinstalled(pkg)]
    if needed:
        utils.install_packages(StrVector(needed))

    # load packages
    lme4 = importr("lme4")
    emmeans = importr("emmeans")

    # simple smoke test
    _ = r("R.version.string")
    R_AVAILABLE = True
    print("R bridge ready.")
except Exception as e:
    print("R bridge init failed:", e)
    R_AVAILABLE = False

In [ ]:
# Paths
data_path = os.path.join(os.getcwd(), 'data/annotations')
image_path = os.path.join(os.getcwd(), 'data/images')
output_path = os.path.join(os.getcwd(), 'outputs')

os.makedirs(output_path, exist_ok=True)


In [ ]:
# Configs 
ACTIVE_MODELS_KEYS = ["GPT", "Claude", "Llama", "Gemini"]
PROMPT_VARIATIONS_KEYS = ["With Monk Scale"] 
CONTEXT_CONDITIONS_KEYS = [
    "baseline", "ignore_world", "embrace_world", "cultural_white",
    "cultural_black", "cultural_asain", "cultural_hispanic",
    "cultural_indigenous", "cultural_africanamerican", "cultural_colorism",
    "cultural_lighter", "cultural_darker"
]

    # Define color mapping for model families
model_colors = {
        'GPT': '#2ecc71',    # Green
        'Claude': '#e74c3c', # Red
        'Llama': '#3498db',  # Blue
        'Gemini': '#f1c40f', # Yellow
    }

## 1. Loading Data & Parsing

In [ ]:
raw_annotations = pd.read_csv(os.path.join(data_path, 'WITHMONKSCALE_concatenated_FINALdata.csv'))

print("Data Shape:", raw_annotations.shape)
print("Columns:", raw_annotations.columns.tolist())
print("First few rows of data:")
print(raw_annotations.head())

In [ ]:
df = raw_annotations.copy()
# Parse the LLM responses

def parse_llm_response(text):
    """
    Parse an LLM response text based on the format:
    Monk Rating: [1–10]
    Confidence: [1–7]
    Rationale: [up to 40 words]

    Also handles "Monk:" or "Rating:" for scale for flexibility with older analysis.py.
    The new primary format is "Monk Rating:".
    
    Returns:
        (scale_numeric, confidence, rationale)
        If parsing fails, returns (None, None, None).
    """
    if not isinstance(text, str):
        return (None, None, None)

    scale_numeric = None
    confidence = None
    rationale = None

    # Try parsing the specific "Monk Rating:" format first
    scale_match_specific = re.search(r'Monk Rating:\s*(\d+)', text, re.IGNORECASE)
    if scale_match_specific:
        try:
            scale_numeric = int(scale_match_specific.group(1))
        except ValueError:
            scale_numeric = None
    else:
        # Fallback to more general patterns if specific one not found
        # Handles "Monk: X" or "Rating: X"
        scale_pattern_general = r'(?:Monk:\s*(\d+)|Rating:\s*(\d+))'
        scale_match_general = re.search(scale_pattern_general, text, re.IGNORECASE)
        if scale_match_general:
            if scale_match_general.group(1): # Monk: X
                try:
                    scale_numeric = int(scale_match_general.group(1))
                except ValueError:
                    scale_numeric = None
            elif scale_match_general.group(2): # Rating: X
                try:
                    scale_numeric = int(scale_match_general.group(2))
                except ValueError:
                    scale_numeric = None
    
    # Extract confidence (e.g., "Confidence: 3" or "Confidence: (3)")
    conf_pattern = r'Confidence:\s*\(?(\d+)\)?'
    conf_match = re.search(conf_pattern, text, re.IGNORECASE)
    if conf_match:
        try:
            confidence = int(conf_match.group(1))
        except ValueError:
            confidence = None

    # Extract rationale (everything after "Rationale:")
    rationale_pattern = r'Rationale:\s*(.*)'
    rat_match = re.search(rationale_pattern, text, re.IGNORECASE | re.DOTALL)
    if rat_match:
        rationale = rat_match.group(1).strip()

    return (scale_numeric, confidence, rationale)

def build_long_df(df: pd.DataFrame, ACTIVE_MODELS_KEYS: list, CONTEXT_CONDITIONS_KEYS: list) -> pd.DataFrame:
    """
    Convert wide-format raw response columns into long-format DataFrame.
    Expects 'image_name' and raw columns named MODEL_Variation_Condition.
    Handles conditions with underscores.
    """
    records = []
    for col in df.columns:
        if col in ('image_name', 'image_path'):
            continue
        parts = col.split('_', 2) 
        if len(parts) < 3:
            continue
        model, condition = parts[0], parts[2]
        if model not in ACTIVE_MODELS_KEYS or condition not in CONTEXT_CONDITIONS_KEYS:
            continue
        # Parse the LLM response
        # Parse all rows; do NOT drop NaNs
        parsed_series = df[col].apply(parse_llm_response)

        for idx, parsed in parsed_series.items():
            # Default to NaNs
            rating = np.nan
            confidence = np.nan
            rationale = np.nan

            # Accept various parse outputs
            if isinstance(parsed, tuple):
                # Allow (rating,), (rating, conf), or (rating, conf, rationale)
                if len(parsed) >= 1: rating = parsed[0]
                if len(parsed) >= 2: confidence = parsed[1]
                if len(parsed) >= 3: rationale = parsed[2]
            elif parsed is not None and not (isinstance(parsed, float) and pd.isna(parsed)):
                # If parser returned a bare rating
                rating = parsed

            records.append({
                'image_name': df.at[idx, 'image_name'],
                'model': model,
                'condition': condition,
                'rating': rating,
                'confidence': confidence,
                'rationale': rationale
            })

    long_df = pd.DataFrame(records)
    long_df['rating'] = pd.to_numeric(long_df['rating'], errors='coerce')
    long_df['confidence'] = pd.to_numeric(long_df['confidence'], errors='coerce')
    # print the unique values of model and condition
    print(f"Unique models: {long_df['model'].unique()}")
    print(f"Unique personas: {long_df['condition'].unique()}")
    return long_df
long_df = build_long_df(df, ACTIVE_MODELS_KEYS, CONTEXT_CONDITIONS_KEYS)

print("Long DataFrame Shape:", long_df.shape)
print("First few rows of long DataFrame:")
print(long_df.head())

In [ ]:
# QC -- images in long_df vs images directory
image_files = os.listdir(image_path)
image_names = [os.path.splitext(f)[0] for f in image_files if f.endswith(('.jpg', '.png'))]
print("Unique image names in images directory:", image_names)

unique_images = long_df['image_name'].unique()
unique_images = [os.path.splitext(img)[0] for img in unique_images]
print("Unique image names after removing extensions:", unique_images)
missing_images = set(unique_images) - set(image_names)
if missing_images:
    print("Missing images in the images directory:", missing_images)
else:
    print("All images in long_df are present in the images directory.")

In [ ]:
# REJECTION RATES 
# 1) Find rows with NaN ratings
nan_mask = long_df['rating'].isna()
nan_rows = long_df[nan_mask].copy()

# 2) Quick summary
print(f"NaN rating rows: {nan_rows.shape[0]} of {long_df.shape[0]} total")
print("Examples:\n", nan_rows.head())

In [ ]:
# drop all images that have one or more NaN ratings
images_with_nan = nan_rows['image_name'].unique()
print(f"Images with NaN ratings: {len(images_with_nan)}")

# Drop these images from the long_df
long_df_balanced = long_df[~long_df['image_name'].isin(images_with_nan)]
print(f"Long DataFrame shape after dropping NaN images: {long_df_balanced.shape}")
long_df_balanced.to_csv(os.path.join(output_path, 'long_df_balanced.csv'), index=False)

In [ ]:
# Save the long DataFrame to a CSV file drop NaNs in ratings
long_df = long_df.dropna(subset=['rating'])
print(f"Long DataFrame shape after dropping NaN ratings: {long_df.shape}")
csv_output_path = os.path.join(output_path, 'long_df.csv')
long_df.to_csv(csv_output_path, index=False)
print(f"Long DataFrame saved to {output_path}")

## 2. Inter- and Intra- Annotator Analysis 

### i. Krippendorff Alpha

In [ ]:
def _alpha_from_pivot(pivot: pd.DataFrame) -> float:
    """
    pivot: rows=items (image_name), cols=raters, values=ratings
    Missing ratings should be NaN.
    """
    # Drop items with no data at all
    pivot = pivot.dropna(how='all')
    # Need at least 2 raters and 2 items with some data
    if pivot.shape[0] < 2 or pivot.shape[1] < 2:
        return np.nan
    data = pivot.to_numpy(dtype=float).T  # raters as rows
    try:
        return krippendorff.alpha(reliability_data=data, level_of_measurement='interval')
    except Exception:
        return np.nan

def calculate_krippendorff_alpha(
    long_df: pd.DataFrame,
    mode: str = "all",              # "all", "by_persona", "within_model"
    aggfunc = "mean"
) -> Union[float, Dict[str, float]]:
    """
    long_df must have columns: 'image_name', 'model', 'condition', 'rating'
    mode:
      - "all": one alpha across all models (raters=models)
      - "by_persona": alpha per condition/persona (raters=models within each condition)
      - "within_model": alpha per model (raters=conditions/personas within each model)
    aggfunc: how to combine duplicates per (image, rater). Usually "mean".
    """
    required = {"image_name", "model", "condition", "rating"}
    missing = required - set(long_df.columns)
    if missing:
        raise KeyError(f"long_df missing required columns: {missing}")

    if mode == "all":
        pivot = pd.pivot_table(
            long_df, index="image_name", columns="model",
            values="rating", aggfunc=aggfunc, dropna=False
        )
        return _alpha_from_pivot(pivot)

    elif mode == "by_persona":
        out: Dict[str, float] = {}
        for persona, g in long_df.groupby("condition", dropna=False):
            pivot = pd.pivot_table(
                g, index="image_name", columns="model",
                values="rating", aggfunc=aggfunc, dropna=False
            )
            out[persona] = _alpha_from_pivot(pivot)
        return out

    elif mode == "within_model":
        out: Dict[str, float] = {}
        for mdl, g in long_df.groupby("model", dropna=False):
            # Now raters are personas (conditions)
            pivot = pd.pivot_table(
                g, index="image_name", columns="condition",
                values="rating", aggfunc=aggfunc, dropna=False
            )
            out[mdl] = _alpha_from_pivot(pivot)
        return out

    else:
        raise ValueError("mode must be one of: 'all', 'by_persona', 'within_model'")


In [ ]:
# run the Krippendorff's alpha calculation
alpha_all = calculate_krippendorff_alpha(long_df, mode="all")
print(f"Krippendorff's alpha (all models): {alpha_all:.4f}")

alpha_by_persona = calculate_krippendorff_alpha(long_df, mode="by_persona")
print("Krippendorff's alpha by persona:")
for persona, alpha in alpha_by_persona.items():
    print(f"{persona}: {alpha:.4f}")

alpha_within_model = calculate_krippendorff_alpha(long_df, mode="within_model")
print("Krippendorff's alpha within each model:")
for model, alpha in alpha_within_model.items():
    print(f"{model}: {alpha:.4f}")

### ii. ICC 

In [ ]:
def compute_icc(long_df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute Intraclass Correlation Coefficient (ICC) for rating agreement.
    """
    df = long_df.pivot_table(index='image_name', columns='model', values='rating')
    df = df.reset_index().melt(id_vars='image_name', var_name='model', value_name='rating')
    df['image_name'] = df['image_name'].astype(str)  # Required for pingouin
    return intraclass_corr(data=df, targets='image_name', raters='model', ratings='rating')

In [ ]:
icc_results = compute_icc(long_df)
print("Intraclass Correlation Coefficient (ICC) results comparing models:")
print(icc_results)

In [ ]:

def compute_icc_by_persona(long_df: pd.DataFrame,
                           min_raters: int = 2,
                           balanced: bool = False,
                           nan_policy: str = 'omit') -> pd.DataFrame:
    """
    ICC per condition/persona with models as raters.
    - If balanced=False (default), uses nan_policy='omit' so unbalanced panels are OK.
    - If balanced=True, keeps only images rated by ALL models within each persona.
    """
    # 1) average duplicates so each (image, condition, model) is a single value
    base = (long_df
            .groupby(['image_name', 'condition', 'model'], dropna=False, as_index=False)['rating']
            .apply(lambda s: np.nan if s.isna().all() else s.mean()))
    base['rating'] = pd.to_numeric(base['rating'], errors='coerce')
    base['image_name'] = base['image_name'].astype(str)

    out = []
    for cond, g in base.groupby('condition', dropna=False):
        if balanced:
            # keep only images with a rating from EVERY model in this persona
            piv = g.pivot_table(index='image_name', columns='model', values='rating', aggfunc='mean')
            piv = piv.dropna(axis=0)  # drop any image with a missing model
            if piv.shape[0] < 2 or piv.shape[1] < 2:
                continue
            g2 = piv.reset_index().melt(id_vars='image_name', var_name='model', value_name='rating')
            g2 = g2.dropna(subset=['rating'])
            if g2['image_name'].nunique() < 2 or g2['model'].nunique() < 2:
                continue
            res = intraclass_corr(data=g2, targets='image_name', raters='model', ratings='rating')
        else:
            # allow unbalanced: keep images with at least `min_raters` ratings, let PG handle the rest
            ok_items = g.groupby('image_name')['rating'].apply(lambda s: s.notna().sum() >= min_raters)
            g2 = g[g['image_name'].isin(ok_items[ok_items].index)].copy()
            g2 = g2.dropna(subset=['rating'])
            if g2['image_name'].nunique() < 2 or g2['model'].nunique() < 2:
                continue
            # key change vs your version -> allow unbalanced
            res = intraclass_corr(data=g2, targets='image_name', raters='model',
                                  ratings='rating', nan_policy=nan_policy)

        res.insert(0, 'condition', cond)
        out.append(res)

    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()


In [ ]:
# Unbalanced allowed (recommended; mirrors your working global ICC)
icc_by_persona = compute_icc_by_persona(long_df, balanced=False, nan_policy='omit')
print(icc_by_persona)

# Strict sensitivity: only images rated by ALL models within each persona
icc_by_persona_bal = compute_icc_by_persona(long_df, balanced=True)
print(icc_by_persona_bal)


## 3. Statistical Analysis (LRT, EMMs, Pairwise Contrasts)

In [ ]:
long_df = pd.read_csv(os.path.join(output_path, 'long_df.csv'))

### i. LRT

In [ ]:

def fit_mixed_models(long_df: pd.DataFrame):
    """
    Fit LMMs: full (model*condition), model-only, condition-only, and null.
    """
    for col in ('model', 'condition', 'image_name'):
        long_df[col] = long_df[col].astype('category')
    md_full = smf.mixedlm('rating ~ model * condition', long_df, groups=long_df['image_name'])
    res_full = md_full.fit(reml=False)
    md_model = smf.mixedlm('rating ~ model', long_df, groups=long_df['image_name'])
    res_model = md_model.fit(reml=False)
    md_cond = smf.mixedlm('rating ~ condition', long_df, groups=long_df['image_name'])
    res_cond = md_cond.fit(reml=False)
    md_null = smf.mixedlm('rating ~ 1', long_df, groups=long_df['image_name'])
    res_null = md_null.fit(reml=False)
    return res_full, res_model, res_cond, res_null

def lrt(res_reduced, res_full):
    stat = 2 * (res_full.llf - res_reduced.llf)
    df_diff = res_full.df_modelwc - res_reduced.df_modelwc
    p = chi2.sf(stat, df_diff)
    return stat, df_diff, p


def run_lrt_tests(res_full, res_model, res_cond, res_null) -> pd.DataFrame:
    tests = [
        ('Null vs Full',    res_null,  res_full),
        ('Null vs Model',   res_null,  res_model),
        ('Null vs Condition',res_null, res_cond),
        ('Model vs Full',   res_model, res_full),
        ('Condition vs Full',res_cond, res_full),
    ]
    rows = []
    for name, base, full in tests:
        stat, df_diff, p = lrt(base, full)
        rows.append({'Test': name, 'Chi2': stat, 'DF': df_diff, 'p-value': p})
    return pd.DataFrame(rows)



In [ ]:
# # Run LRTs across models
res_full_all, res_model_all, res_cond_all, res_null_all = fit_mixed_models(long_df)
cross_lrt_df = run_lrt_tests(res_full_all, res_model_all, res_cond_all, res_null_all)
print("Cross-model LRT results:")
print(cross_lrt_df)


In [ ]:

def fit_ordinal_mixed_model_r(long_df: pd.DataFrame):
    """
    Fit ordinal mixed-effects model using R's ordinal::clmm.
    """
    if not R_AVAILABLE:
        raise RuntimeError('R and required packages (ordinal) are not available')

    from rpy2.robjects import r

    # Push to R
    with localconverter(pandas2ri.converter):
        ro.globalenv['rdf'] = pandas2ri.py2rpy(long_df)

    # Make sure R understands as ordered factor
    r('rdf$rating_factor <- factor(rdf$rating, ordered=TRUE)')
    r('rdf$model <- as.factor(rdf$model)')
    r('rdf$condition <- as.factor(rdf$condition)')
    r('rdf$image_name <- as.factor(rdf$image_name)')

    # Fit ordinal mixed model
    print("Fitting ordinal mixed-effects model...")
    r('library(ordinal)')
    r('ord_fit <- clmm(rating_factor ~ model * condition + (1|image_name), data=rdf)')
    # Text summary
    summary_text = "\n".join(list(r('capture.output(summary(ord_fit))')))

    # Fixed-effects table as a pandas DataFrame
    with localconverter(pandas2ri.converter):
        coefs_df = r('as.data.frame(coef(summary(ord_fit)))')

    return summary_text, coefs_df

summary_test, coefs_df = fit_ordinal_mixed_model_r(long_df)
print("Ordinal mixed-effects model summary:")
print(summary_test)

In [ ]:
import pandas as pd

def format_clmm_table_for_supplement(coefs_df: pd.DataFrame, save_path="Table_S5_CLMM.csv"):
    """
    Clean, format, and export CLMM coefficients for Nature Medicine–style Supplementary Table S5.
    Adds significance stars, bolds p<0.05 rows, and interprets direction of effects.
    """
    # --- Rename columns for clarity
    coefs_df = coefs_df.rename(columns={
        "Estimate": "β (Estimate)",
        "Std. Error": "Std. Error",
        "z value": "z",
        "Pr(>|z|)": "p-value"
    })

    # --- Filter out threshold rows (e.g., "2|3", "3|4")
    coefs_df = coefs_df[~coefs_df.index.str.contains(r"\|")].copy()

    # --- Ensure numeric columns are numeric
    numeric_cols = ["β (Estimate)", "Std. Error", "z", "p-value"]
    coefs_df[numeric_cols] = coefs_df[numeric_cols].apply(pd.to_numeric, errors="coerce")

    # --- Significance stars
    def p_to_stars(p):
        if p < 0.001: return '***'
        elif p < 0.01: return '**'
        elif p < 0.05: return '*'
        else: return ''
    coefs_df["Significance"] = coefs_df["p-value"].apply(p_to_stars)

    # --- Direction of effect
    coefs_df["Direction"] = coefs_df["β (Estimate)"].apply(
        lambda b: "Darker rating (positive)" if b > 0 else "Lighter rating (negative)"
    )

    # --- Round for readability
    coefs_df["β (Estimate)"] = coefs_df["β (Estimate)"].round(2)
    coefs_df["Std. Error"] = coefs_df["Std. Error"].round(2)
    coefs_df["z"] = coefs_df["z"].round(2)
    coefs_df["p-value_display"] = coefs_df["p-value"].apply(
        lambda p: f"{p:.2e}" if p < 0.001 else round(p, 3)
    )

    # --- Bold significant rows (for Markdown/HTML outputs)
    def bold_if_sig(term, p):
        return f"**{term}**" if p < 0.05 else term
    coefs_df["Term_display"] = [
        bold_if_sig(term, p) for term, p in zip(coefs_df.index, coefs_df["p-value"])
    ]

    # --- Reorder columns for clarity
    final_cols = [
        "Term_display", "β (Estimate)", "Std. Error", "z",
        "p-value_display", "Significance", "Direction"
    ]
    formatted_df = coefs_df[final_cols].rename(columns={
        "Term_display": "Term",
        "p-value_display": "p-value"
    })

    # --- Save CSV for supplement
    formatted_df.to_csv(save_path, index=False)
    print(f"Supplementary Table S5 saved to {save_path}")

    return formatted_df

# Example usage:
supp_table = format_clmm_table_for_supplement(coefs_df)
supp_table.head()


In [ ]:
from docx import Document
from docx.shared import Inches
import pandas as pd

def export_clmm_table_to_word(df, save_path="Table_S5_CLMM.docx"):
    doc = Document()
    doc.add_heading("Supplementary Table S5", level=1)
    doc.add_paragraph(
        "Coefficients from the ordinal cumulative-link mixed model (rating ~ model × persona + (1 | image)). "
        "Positive β estimates indicate darker predicted skin-tone ratings (higher Monk scale values), "
        "whereas negative estimates indicate lighter predicted ratings. "
        "The model used a logit link and included random intercepts for image (n = 72, SD = 1.43). "
        "*p < 0.05; **p < 0.01; ***p < 0.001.*"
    )

    # Add table
    table = doc.add_table(rows=1, cols=len(df.columns))
    table.style = "Table Grid"

    # Add header row
    hdr_cells = table.rows[0].cells
    for i, col_name in enumerate(df.columns):
        hdr_cells[i].text = col_name

    # Add data rows
    for _, row in df.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            text = str(val)
            # Bold significant rows
            if row["Significance"] in ["*", "**", "***"]:
                run = row_cells[i].paragraphs[0].add_run(text)
                run.bold = True
            else:
                row_cells[i].text = text

    doc.save(save_path)
    print(f"Word file saved: {save_path}")

formatted_df = format_clmm_table_for_supplement(coefs_df)
export_clmm_table_to_word(formatted_df)


In [ ]:
import re
import pandas as pd

# ---- Configuration helpers ----
MODEL_MAP = {
    "modelGPT": "GPT",
    "modelGemini": "Gemini",
    "modelLlama": "Llama",
    # Claude is the reference (intercept) and thus absent as a main effect
}
# Optional tidy-up for condition labels
CONDITION_RENAMES = {
    "cultural_asain": "cultural_asian",  # fix typo if present in R output
}

def sig_stars(p):
    try:
        p = float(p)
    except Exception:
        return ""
    if p < 0.001: return "***"
    if p < 0.01:  return "**"
    if p < 0.05:  return "*"
    if p < 0.1:   return "·"
    return ""

def effect_direction(beta):
    try:
        b = float(beta)
    except Exception:
        return ""
    # Positive beta => higher (darker) Monk rating; Negative => lower (lighter)
    return "Darker (β>0)" if b > 0 else "Lighter (β<0)"

def prettify_condition(raw):
    # Remove leading prefix and normalize underscores/spaces
    c = raw
    c = CONDITION_RENAMES.get(c, c)
    c = re.sub(r"^cultural_", "", c)
    c = c.replace("_", " ")
    # small capitalizations
    c = c.replace("embrace world", "embrace world-knowledge")
    c = c.replace("ignore world", "ignore world-knowledge")
    return c

def categorize_terms(coefs_df: pd.DataFrame):
    """
    coefs_df is the result of:
      coefs_df = r('as.data.frame(coef(summary(ord_fit)))')
    Row names contain term labels (e.g., 'modelGPT', 'conditioncultural_darker', 'modelLlama:conditioncultural_lighter')
    """
    df = coefs_df.copy()

    # Ensure term labels are available as a column
    if df.index.name is None or df.index.name == "":
        df.index.name = "Term"
    df.reset_index(inplace=True)

    # Standardize column names coming from R
    rename_map = {
        "Estimate": "beta",
        "Std. Error": "SE",
        "z value": "z",
        "Pr(>|z|)": "p"
    }
    df.rename(columns=rename_map, inplace=True)

    # Work on numeric copies for sorting/formatting later
    for col in ["beta", "SE", "z", "p"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Categorize & extract parts
    term_type = []
    model_label = []
    persona_label = []

    for t in df["Term"]:
        if ":" in t:
            term_type.append("Model×Persona interaction")
            m, c = t.split(":", 1)
            model_label.append(MODEL_MAP.get(m, m.replace("model", "")))
            c = c.replace("condition", "")
            persona_label.append(prettify_condition(c))
        elif t.startswith("model"):
            term_type.append("Model main effect")
            model_label.append(MODEL_MAP.get(t, t.replace("model", "")))
            persona_label.append("")
        elif t.startswith("condition"):
            term_type.append("Persona main effect (Claude ref.)")
            model_label.append("Claude (ref.)")
            c = t.replace("condition", "")
            persona_label.append(prettify_condition(c))
        else:
            term_type.append("Other")
            model_label.append("")
            persona_label.append("")

    df["Category"] = term_type
    df["Model"] = model_label
    df["Persona"] = persona_label

    # Add interpretation columns
    df["Direction"] = df["beta"].apply(effect_direction)
    df["Sig"] = df["p"].apply(sig_stars)

    # Pretty numeric formatting for supplement
    def fmt(x, dec=2):
        if pd.isna(x): return ""
        # show very small p-values as scientific
        if isinstance(x, float) and (x < 0.001) and (x >= 0):
            return f"{x:.2e}"
        return f"{x:.{dec}f}"

    df["β (Estimate)"] = df["beta"].apply(fmt)
    df["Std. Error"] = df["SE"].apply(fmt)
    df["z"] = df["z"].apply(fmt)
    df["p-value"] = df["p"].apply(lambda x: "<0.001" if (pd.notna(x) and x < 0.001) else fmt(x, 3))

    # Arrange columns
    cols = [
        "Category", "Model", "Persona", "Term",
        "β (Estimate)", "Std. Error", "z", "p-value", "Sig", "Direction"
    ]
    df_out = df[cols].copy()

    # Sort within categories by p-value ascending
    df_out["p_sort"] = pd.to_numeric(df["p"], errors="coerce")
    df_out.sort_values(by=["Category", "p_sort"], inplace=True)
    df_out.drop(columns=["p_sort"], inplace=True)

    return df_out

def export_grouped_tables(df_out: pd.DataFrame, base_path="supp_table_CLMM"):
    """Save overall and per-category CSVs for the supplement."""
    df_out.to_csv(f"{base_path}_all.csv", index=False)
    for cat, sub in df_out.groupby("Category"):
        safe = re.sub(r"[^A-Za-z0-9]+", "_", cat).strip("_").lower()
        sub.to_csv(f"{base_path}_{safe}.csv", index=False)
    print("Saved:")
    print(f" - {base_path}_all.csv")
    print(f" - {base_path}_model_main_effect.csv")
    print(f" - {base_path}_persona_main_effect_claude_ref_.csv")
    print(f" - {base_path}_model_persona_interaction.csv")

# ---- Usage (after you already computed coefs_df) ----
df_out = categorize_terms(coefs_df)
export_grouped_tables(df_out, base_path="supp_table_CLMM")
df_out  # preview


### ii. EMMs

In [ ]:
def compute_emmeans_r(long_df: pd.DataFrame):
    """
    Compute EMMs and pairwise contrasts via R's lme4 + emmeans.
    Returns dict with 'emms' and 'contrasts' DataFrames, with model & condition as strings.
    """
    if not R_AVAILABLE:
        raise RuntimeError('R and required packages (lme4, emmeans) are not available')

    with localconverter(pandas2ri.converter):
        ro.globalenv['rdf'] = pandas2ri.py2rpy(long_df)
    # Ensure factors in R
    ro.r('rdf$condition <- as.factor(rdf$condition)')
    ro.r('rdf$image_name <- as.factor(rdf$image_name)')
    ro.r('rdf$model <- as.factor(rdf$model)')
    # Choose formula and EMM basis
    n_models = long_df['model'].nunique()
    n_conditions = long_df['condition'].nunique()
    if n_models > 1 and n_conditions > 1:
        r_formula, emm_by = 'rating ~ model * condition', '~ model | condition'
    elif n_models > 1:
        #r_formula, emm_by = 'rating ~ model', '~ model'
        r_formula, emm_by = 'rating ~ condition * model', '~ condition | model' # Edies change for interaction effects "does the persona effect depend on the model"
    else:
        r_formula, emm_by = 'rating ~ condition', '~ condition'
    # Fit the mixed model in R
    ro.r(f"fit <- lme4::lmer({r_formula} + (1|image_name), data=rdf)")
    # Compute emmeans
    ro.r(f"emm <- emmeans::emmeans(fit, {emm_by})")
    # Convert to data frame and coerce factors to strings
    ro.r('emm_df <- as.data.frame(emm)')
    ro.r('if("model" %in% colnames(emm_df)) emm_df$model <- as.character(emm_df$model)')
    ro.r('emm_df$condition <- as.character(emm_df$condition)')
    # Compute pairwise contrasts
    ro.r('cont_df <- as.data.frame(contrast(emm, method="pairwise", adjust="holm"))')
    ro.r('if("model" %in% colnames(cont_df)) cont_df$model <- as.character(cont_df$model)')

    # condition-by-model EMMs & contrasts (persona contrasts within each model)
    ro.r('emm_cond_by_model <- emmeans::emmeans(fit, ~ condition | model)')
    ro.r('cont_cond_by_model <- as.data.frame(emmeans::contrast(emm_cond_by_model, '
         '                                      method="pairwise", adjust="holm"))')
    # ADD these two lines:
    ro.r('cont_cond_by_model$contrast <- as.character(cont_cond_by_model$contrast)')
    ro.r('cont_cond_by_model$model <- as.character(cont_cond_by_model$model)')

    # Convert back to pandas
    with localconverter(pandas2ri.converter):
        py_emms = r('emm_df')
        py_cont = r('cont_df')
        py_cont_cond_by_model = r('cont_cond_by_model')
    return {'emms': py_emms, 'contrasts': py_cont, 'contrasts_cond_by_model': py_cont_cond_by_model}

In [ ]:
# Compute EMMs + contrasts across models
res = compute_emmeans_r(long_df)
emm_df = res['emms']
contr_df = res['contrasts']
cross_emms_path = os.path.join(output_path, 'cross_model_emms.csv')
cross_cont_path = os.path.join(output_path, 'cross_model_contrasts.csv')
emm_df.to_csv(cross_emms_path, index=False)
contr_df.to_csv(cross_cont_path, index=False)
print(f'Saved cross-model EMMs to {cross_emms_path}')
print(f'Saved cross-model contrasts to {cross_cont_path}')

In [ ]:
def summarize_contrasts(contr_df, alpha=0.05):
    out = []
    for cond, g in contr_df.groupby('condition'):
        sig = g[g['p.value'] < alpha].copy()
        sig['CI_low']  = sig['estimate'] - 1.96 * sig['SE']
        sig['CI_high'] = sig['estimate'] + 1.96 * sig['SE']
        out.append((cond, sig.sort_values('p.value')[['contrast','estimate','SE','CI_low','CI_high','z.ratio','p.value']]))
    return out

# usage
for cond, tbl in summarize_contrasts(contr_df):
     print(f"\n[{cond}] significant pairwise model differences:")
     print(tbl.to_string(index=False))


In [ ]:


def make_persona_pair_table(contr_cbm: pd.DataFrame, alpha=0.05, min_models=3, add_ci=True):
    """
    Build 'Table 2' of persona-vs-persona contrasts within each model
    from emmeans(fit, ~ condition | model).

    - Keeps only persona pairs that are significant (Holm-adjusted p) in >= min_models models.
    - Final table includes ONLY rows that are themselves significant (p < alpha).
    - Optionally adds 95% CIs (normal approx).
    """
    # Normalize names from emmeans
    df = contr_cbm.rename(columns={
        'p.value': 'p_value',
        'SE': 'SE',
        'estimate': 'estimate',
        'model': 'Model',
        'contrast': 'contrast',
    }).copy()

    # Required columns
    needed = {'Model', 'contrast', 'estimate', 'SE', 'p_value'}
    missing = needed - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns from contrasts_cond_by_model: {missing}")

    # Types
    df['contrast'] = df['contrast'].astype(str)
    df['Model']    = df['Model'].astype(str)
    df['p_value']  = pd.to_numeric(df['p_value'], errors='coerce')
    df['estimate'] = pd.to_numeric(df['estimate'], errors='coerce')
    df['SE']       = pd.to_numeric(df['SE'], errors='coerce')

    # Persona-pair label (emmeans uses "A - B")
    df['Contrast'] = df['contrast'].str.replace(' - ', ' vs ', regex=False)

    # Mark significance
    df['sig'] = df['p_value'] < alpha

    # Count how many UNIQUE models are significant for each persona pair
    sig_counts = (df[df['sig']]
                  .groupby('Contrast')['Model']
                  .nunique()
                  .reset_index(name='models_sig'))

    # Keep persona pairs with >= min_models significant models
    keep_pairs = set(sig_counts.loc[sig_counts['models_sig'] >= min_models, 'Contrast'])

    # Filter to kept pairs AND only significant rows
    out = df[(df['Contrast'].isin(keep_pairs)) & (df['sig'])].copy()

    # Optional 95% CIs (z approx)
    if add_ci:
        z = 1.96
        out['CI_low']  = out['estimate'] - z * out['SE']
        out['CI_high'] = out['estimate'] + z * out['SE']

    # Final columns / order
    cols = ['Contrast', 'Model', 'estimate', 'SE', 'p_value']
    if add_ci:
        cols = ['Contrast', 'Model', 'estimate', 'SE', 'CI_low', 'CI_high', 'p_value']

    out = (out[cols]
           .rename(columns={'estimate':'Estimate', 'p_value':'p-value', 'SE':'SE'})
           .sort_values(['Contrast', 'Model'])
           .reset_index(drop=True))

    # Formatting
    for c in ['Estimate', 'SE']:
        out[c] = out[c].round(2)
    if add_ci:
        out['CI_low']  = out['CI_low'].round(2)
        out['CI_high'] = out['CI_high'].round(2)
    out['p-value'] = out['p-value'].apply(lambda x: f"{float(x):.3f}" if pd.notnull(x) else "NA")

    return out



contr_cbm = res['contrasts_cond_by_model']
table2 = make_persona_pair_table(contr_cbm, alpha=0.05)
table2.to_csv('table2_persona_pairwise_contrasts.csv', index=False)
print(table2.head())

In [ ]:
# Plotting EMMs

def plot_emms_r(emms_df: pd.DataFrame, save_path: str = None, model_name: str = None):
    import matplotlib.pyplot as plt
    import numpy as np

    plt.figure(figsize=(15, 6))
    
    # Define color mapping for model families
    model_colors = {
        'GPT': '#2ecc71',    # Green
        'Claude': '#e74c3c', # Red
        'Llama': '#3498db',  # Blue
        'Gemini': '#f1c40f', # Yellow
    }

    # Mapping for cleaned persona names
    condition_map = {
        'cultural_africanamerican': 'African American',
        'cultural_asain': 'Asian',
        'cultural_hispanic': 'Hispanic',
        'cultural_indigenous': 'Indigenous',
        'cultural_colorism': 'Experienced Colorism',
        'cultural_white': 'White',
        'baseline': 'Baseline',
        'cultural_black': 'Black',
        'cultural_lighter': 'Grew up in a Lighter Skin Toned Community',
        'cultural_darker': 'Grew up in a Darker Skin Toned Community',
        'embrace_world': 'Embrace all World Knowledge',
        'ignore_world': 'Ignore all World Knowledge',
    }
    emms_df['condition_clean'] = emms_df['condition'].map(condition_map).fillna(emms_df['condition'])
    # Build an order with Baseline first, others alphabetical (customize if you want)
    all_conditions = sorted(set(emms_df['condition_clean']) - {'Baseline'})
    persona_order = ['Baseline'] + all_conditions

    # Make it an ordered categorical so sorting respects this order
    emms_df['condition_clean'] = pd.Categorical(
        emms_df['condition_clean'],
        categories=persona_order,
        ordered=True
    )
    print("Unique conditions after mapping:", emms_df['condition_clean'].unique())
    if model_name == 'All Models':
        # Plot for All Models
        fig, ax = plt.subplots(figsize=(15, 6))
        emms_df = emms_df.sort_values(['model', 'condition_clean'])
        models = emms_df['model'].unique()
        x_positions = []
        labels = []
        x = 0
        model_boundaries = []

        for model in models:
            model_data = emms_df[emms_df['model'] == model]
            start_x = x
            for _, row in model_data.iterrows():
                model_family = next((family for family in model_colors if family.lower() in row['model'].lower()), 'Other')
                color = model_colors.get(model_family, '#95a5a6')
                ax.errorbar(x, row['emmean'], yerr=row['SE'], fmt='o', capsize=5, color=color)
                labels.append(row['condition_clean'])
                x_positions.append(x)
                x += 1
            end_x = x - 1
            mid_x = (start_x + end_x) / 2
            model_boundaries.append((x, mid_x, model))
            x += 1  # gap

        # X/Y Labels and ticks
        ax.set_xticks(x_positions)
        ax.set_xticklabels(labels, rotation=45, ha='right')
        #ax.set_ylabel('Estimated Marginal Mean (EMM)')
        ax.set_ylabel('EMM of Monk Skin Tone Rating (1-10)')
        ax.set_title('Estimated Marginal Means (EMM) within Personas Grouped by Model')

        # Add vertical dashed lines between models
        for boundary, _, _ in model_boundaries[:-1]:  # skip last
            ax.axvline(x=boundary - 0.5, color='gray', linestyle='--', alpha=0.3)
        
        # Add model labels at top inside plot
        ax.set_ylim(1, 10)
        ylim = ax.get_ylim()
        y_top = ylim[1] - 0.1 * (ylim[1] - ylim[0])
        for _, mid_x, model in model_boundaries:
            ax.text(mid_x, y_top, model,
                    fontsize=10, fontweight='bold',
                    va='bottom', ha='center',
                    bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))

        #ax.set_ylim(ylim[0], y_top + 0.05 * (ylim[1] - ylim[0]))
        # set y-axis 1-10
        

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.show()

    else:
        # Plot for individual model
        labels = emms_df['condition'].map(condition_map).fillna(emms_df['condition']).tolist()
        x_positions = range(len(labels))
        plt.errorbar(x_positions, emms_df['emmean'], yerr=emms_df['SE'], fmt='o', capsize=5)
        plt.xticks(x_positions, labels, rotation=45, ha='right')
        plt.xlabel('Persona')
        plt.ylabel('EMM')
        title = 'Estimated Marginal Means'
        if model_name:
            title += f' - {model_name}'
        plt.title(title)
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.show()


def plot_emms_by_persona(emms_df: pd.DataFrame, save_path: str = None):
    import matplotlib.pyplot as plt
    import numpy as np

    plt.figure(figsize=(15, 6))
    
    # Define color mapping for model families
    model_colors = {
        'GPT': '#2ecc71',    # Green
        'Claude': '#e74c3c', # Red
        'Llama': '#3498db',  # Blue
        'Gemini': '#f1c40f', # Yellow
    }

    # Mapping for cleaned persona names
    condition_map = {
        'cultural_africanamerican': 'African American',
        'cultural_asain': 'Asian',
        'cultural_hispanic': 'Hispanic',
        'cultural_indigenous': 'Indigenous',
        'cultural_colorism': 'Experienced Colorism',
        'cultural_white': 'White',
        'baseline': 'Baseline',
        'cultural_black': 'Black',
        'cultural_lighter': 'Grew up in a Lighter Skin Toned Community',
        'cultural_darker': 'Grew up in a Darker Skin Toned Community',
        'embrace_world': 'Embrace all World Knowledge',
        'ignore_world': 'Ignore all World Knowledge',
    }
    emms_df['condition_clean'] = emms_df['condition'].map(condition_map).fillna(emms_df['condition'])

    # Sort the dataframe by condition and model
    emms_df = emms_df.sort_values(['condition_clean', 'model'])
    conditions = emms_df['condition_clean'].unique()
    models = emms_df['model'].unique()
    
    x_positions = []
    label_positions = []
    labels = []
    current_x = 0
    persona_boundaries = []

    for condition in conditions:
        condition_data = emms_df[emms_df['condition_clean'] == condition]
        start_x = current_x
        for model in models:
            if model in condition_data['model'].values:
                x_positions.append(current_x)
                current_x += 1
        end_x = current_x - 1
        mid_x = (start_x + end_x) / 2
        label_positions.append(mid_x)
        labels.append(condition)
        persona_boundaries.append((start_x, end_x))
        current_x += 1

    fig, ax = plt.subplots(figsize=(15, 6))
    
    for i, (_, row) in enumerate(emms_df.iterrows()):
        model_family = next((family for family in model_colors.keys()
                             if family.lower() in row['model'].lower()), 'Other')
        color = model_colors.get(model_family, '#95a5a6')
        ax.errorbar(x_positions[i], row['emmean'], yerr=row['SE'], 
                    fmt='o', capsize=5, color=color,
                    label=row['model'] if i == 0 or row['model'] not in [l.get_label() for l in plt.gca().lines] else "")

    # Add vertical dashed lines between personas
    for start_x, end_x in persona_boundaries[:-1]:
        ax.axvline(x=end_x + 0.5, color='gray', linestyle='--', alpha=0.3)
    
    #ax.set_ylim(bottom=ax.get_ylim()[0])  # keep default bottom
    ax.set_ylim(1, 10)  # set y-axis 1-10
    #ax.set_ylabel('Estimated Marginal Mean (EMM)')
    ax.set_ylabel('EMM of Monk Skin Tone Rating (1-10)')
    ax.set_xticks(label_positions)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_title('Estimated Marginal Means (EMM) - Grouped by Persona')

    # Add color-coded legend for model families
    unique_models = emms_df['model'].unique()
    handles = []
    for model in unique_models:
        model_family = next((family for family in model_colors if family.lower() in model.lower()), 'Other')
        color = model_colors.get(model_family, '#95a5a6')
        handles.append(plt.Line2D([0], [0], marker='o', color='w',
                                  markerfacecolor=color, markersize=10, label=model))
    ax.legend(handles=handles, bbox_to_anchor=(1.02, 1), loc='upper left')

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.show()

def plot_emms_superimposed(emms_df: pd.DataFrame, save_path: str = None, model_name: str = 'All Models'):
    """
    Plots EMMs superimposed by persona (x-axis = persona), color-coded by model.
    - All circular points, no connecting lines, aligned on same vertical line per persona.
    - Legend shows dots only (no error bars).
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    from itertools import cycle
    import matplotlib.lines as mlines

    plt.figure(figsize=(12, 6))

    # Color mapping for model families
    model_colors = {
        'GPT': '#2ecc71',    # Green
        'Claude': '#e74c3c', # Red
        'Llama': '#3498db',  # Blue
        'Gemini': '#f1c40f', # Yellow
    }
    default_palette = ['#8e44ad', '#16a085', '#d35400', '#2c3e50', '#7f8c8d']

    # Mapping for cleaned persona names
    condition_map = {
        'cultural_africanamerican': 'African American',
        'cultural_asain': 'Asian',
        'cultural_hispanic': 'Hispanic',
        'cultural_indigenous': 'Indigenous',
        'cultural_colorism': 'Experienced Colorism',
        'cultural_white': 'White',
        'baseline': 'Baseline',
        'cultural_black': 'Black',
        'cultural_lighter': 'Grew up in a Lighter Skin Toned Community',
        'cultural_darker': 'Grew up in a Darker Skin Toned Community',
        'embrace_world': 'Embrace all World Knowledge',
        'ignore_world': 'Ignore all World Knowledge',
    }

    # Create cleaned persona column
    emms_df = emms_df.copy()
    emms_df['condition_clean'] = emms_df['condition'].map(condition_map).fillna(emms_df['condition'])

    # Build persona order: Baseline first, then alphabetical
    all_conditions = sorted(set(emms_df['condition_clean']) - {'Baseline'})
    persona_order = ['Baseline'] + all_conditions
    emms_df['condition_clean'] = pd.Categorical(emms_df['condition_clean'],
                                               categories=persona_order,
                                               ordered=True)

    # Filter if a single model requested
    if model_name != 'All Models':
        emms_df = emms_df[emms_df['model'] == model_name]

    # Determine unique personas and models (keep persona order)
    personas = [p for p in persona_order if p in emms_df['condition_clean'].unique()]
    models = sorted(emms_df['model'].unique())

    if len(models) == 0 or len(personas) == 0:
        raise ValueError("No data to plot: check 'model_name' filter and that 'condition' values exist in the dataframe.")

    # x positions for personas (categorical)
    x_positions = np.arange(len(personas))

    # Prepare color mapping for each model
    model_to_color = {}
    default_cycle = cycle(default_palette)
    for m in models:
        family = next((family for family in model_colors if family.lower() in m.lower()), None)
        model_to_color[m] = model_colors.get(family, next(default_cycle))

    fig, ax = plt.subplots(figsize=(12, 6))

    # Plot each model’s points (all aligned on same x positions)
    for model in models:
        model_df = emms_df[emms_df['model'] == model]
        means, ses = [], []
        for persona in personas:
            row = model_df[model_df['condition_clean'] == persona]
            if not row.empty:
                means.append(float(row['emmean'].iloc[0]))
                ses.append(float(row['SE'].iloc[0]))
            else:
                means.append(np.nan)
                ses.append(np.nan)

        valid = ~np.isnan(means)
        if valid.any():
            ax.errorbar(x_positions[valid], np.array(means)[valid],
                        yerr=np.array(ses)[valid],
                        fmt='o', capsize=4,
                        color=model_to_color[model],
                        markersize=6, alpha=0.9)

    # X-axis labels (only personas)
    ax.set_xticks(x_positions)
    ax.set_xticklabels(personas, rotation=45, ha='right', fontsize=10)

    ax.set_ylabel('EMM of Monk Skin Tone Rating (1–10)')
    ax.set_title('Estimated Marginal Means by Persona — Models Superimposed')

    # Y limits consistent with your 1–10 scale
    ax.set_ylim(1, 10)
    ax.grid(axis='y', linestyle='--', alpha=0.3)

    # --- Custom legend: dots only (no error bars) ---
    legend_handles = [
        mlines.Line2D([], [], color=color, marker='o', linestyle='None', markersize=6, label=model)
        for model, color in model_to_color.items()
    ]
    ax.legend(handles=legend_handles, title='Model',
              bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.show()



In [ ]:
# Plot cross-model EMMs
emms = res["emms"].copy()          # columns should include: model, condition, emmean, SE
contrasts = res["contrasts"].copy()

#save the EMMs and contrasts to CSV files
emms.to_csv(os.path.join(output_path, 'cross_model_emms.csv'), index=False)
contrasts.to_csv(os.path.join(output_path, 'cross_model_contrasts.csv'), index=False)

cross_plot_path = os.path.join(output_path, 'cross_model_emms.png')
plot_emms_r(emms, cross_plot_path, model_name='All Models')
print(f'Saved cross-model EMM plot to {cross_plot_path}')

# # After the original cross-model EMMs plot, add:
# # Plot cross-model EMMs grouped by persona
cross_plot_persona_path = os.path.join(output_path, 'cross_model_emms_by_persona.png')
plot_emms_by_persona(emms, cross_plot_persona_path)
print(f'Saved cross-model EMM plot (grouped by persona) to {cross_plot_persona_path}')

# new superimposed plot
cross_plot_superimposed_path = os.path.join(output_path, 'cross_model_emms_superimposed.png')
plot_emms_superimposed(emms, cross_plot_superimposed_path, model_name='All Models')
print(f'Saved cross-model EMM superimposed plot to {cross_plot_superimposed_path}')


In [ ]:
def persona_consistency_percent_from_emms(emms: pd.DataFrame,
                                          baseline_label: str = "baseline",
                                          require_all_models: bool = True):
    """
    Compute the % of personas where all models' effects vs baseline go in the same direction.
    - emms: DataFrame with columns ['model','condition','emmean'] (from emmeans)
    - baseline_label: value in 'condition' that represents the baseline
    - require_all_models: if True, only evaluate personas that have *all* models present
                          both at baseline and at that persona. If False, allow partial overlap (>=2 models).
    Returns: (percent_consistent, details_df)
    """
    df = emms[['model','condition','emmean']].copy()
    df['model'] = df['model'].astype(str)
    df['condition'] = df['condition'].astype(str)

    # Baseline EMM per model
    base = (df[df['condition'] == baseline_label]
              .rename(columns={'emmean':'baseline_emm'})
              [['model','baseline_emm']])

    # Merge baseline back
    merged = df.merge(base, on='model', how='left')
    merged['effect_vs_base'] = merged['emmean'] - merged['baseline_emm']

    # Exclude the baseline rows from evaluation
    eval_df = merged[merged['condition'] != baseline_label].copy()

    # Helper to decide consistency for one persona
    def persona_consistency(g: pd.DataFrame):
        # drop NaNs and exact zeros (neutral) for direction check
        vals = g['effect_vs_base'].to_numpy(dtype=float)
        vals = vals[np.isfinite(vals)]
        nonzero = vals[~np.isclose(vals, 0.0)]
        # check coverage
        models_present = g['model'].nunique()
        models_needed = base['model'].nunique() if require_all_models else 2
        if models_present < models_needed:
            return 'insufficient'
        if nonzero.size == 0:
            return 'neutral'  # no directional change
        all_pos = np.all(nonzero > 0)
        all_neg = np.all(nonzero < 0)
        return 'consistent' if (all_pos or all_neg) else 'inconsistent'

    # Evaluate per persona
    status = (eval_df.groupby('condition', as_index=False)
                     .apply(persona_consistency)
                     .rename(columns={None:'status'})  # pandas <2.1 behavior
              )
    status.columns = ['condition','status']  # ensure names

    # Compute percentage over evaluable personas
    evaluable = status[status['status'].isin(['consistent','inconsistent','neutral'])]
    if len(evaluable) == 0:
        percent = np.nan
    else:
        percent = 100.0 * (evaluable['status'].eq('consistent').sum() / len(evaluable))

    return percent, status.sort_values('condition')


emms = res['emms']
pct, table = persona_consistency_percent_from_emms(emms, baseline_label="baseline", require_all_models=True)
print(f"Consistent persona effects: {pct:.1f}%")
display(table)


## 4. Qualitative Analysis of LLM Rationales 

In [ ]:
# check long df rational for all models???
long_df = pd.read_csv(os.path.join(output_path, 'long_df.csv'))

# count rationales for each model 
def count_rationales(long_df: pd.DataFrame) -> pd.DataFrame:
    """
    Count the number of rationales provided by each model.
    Assumes 'rationale' column exists in long_df.
    """
    if 'rationale' not in long_df.columns:
        raise KeyError("long_df must contain a 'rationale' column.")
    
    # Create a new column to indicate if a rationale was provided
    long_df['has_rationale'] = long_df['rationale'].notna() & (long_df['rationale'].str.strip() != '')
    
    # Group by model and count rationales
    rationale_counts = (long_df.groupby('model')['has_rationale']
                        .sum()
                        .reset_index()
                        .rename(columns={'has_rationale': 'rationale_count'}))
    
    return rationale_counts
rationale_counts = count_rationales(long_df)
print("Rationale counts by model:")
print(rationale_counts)


In [ ]:
import re, pandas as pd
from sympy import denom
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# -------------------------
# 1) Prep
# -------------------------
def get_rationales(long_df):
    cols = ['model','condition','image_name','rating','rationale']
    df = long_df[cols].copy()
    df['rationale'] = df['rationale'].fillna('').astype(str).str.strip()
    return df[df['rationale']!=''].reset_index(drop=True)

# -------------------------
# 2) Lexicons / detectors
# -------------------------
HEDGES = r'\b(might|may|could|possibly|perhaps|appears|seems|likely|suggests)\b'

# Broadened disclaimer detector
DISCLAIMER = (
    r"\b(as (an )?(ai|language model))\b|"
    r"\b(i\s*(cannot|can't|do not|don't|am unable|i’m unable|i am unable|won't|will not))\b|"
    r"\b(i (don'?t|do not) have (enough )?(info|information|access|the ability))\b|"
    r"\b(i cannot (determine|be certain|provide|diagnose))\b"
)

NORMATIVE = r'\b(should|ought|must|avoid|fair(ness)?|bias(ed|)?|harm(ful)?)\b'

# Persona cue mentions
# Compile with VERBOSE + IGNORECASE for readability/flexibility
RXFLAGS = re.IGNORECASE | re.VERBOSE

PERSONA_PATTERNS = {
    # Knowledge framing personas
    "ignore_world": re.compile(r"""
        \b(?:ignore|disregard|exclude)\s+
           (?:world(?:\s+knowledge)?|external\s+knowledge|prior\s+knowledge)\b
    """, RXFLAGS),
    "embrace_world": re.compile(r"""
        \b(?:embrace|use|leverage|incorporate)\s+
           (?:world(?:\s+knowledge)?|external\s+knowledge|prior\s+knowledge)\b
    """, RXFLAGS),

    # Community-context personas (use "skin tone" phrasing)
    "community_lighter": re.compile(r"""
        \b(?:lighter(?:\s+skin(?:\s*tone)?)?|fair(?:er)?(?:\s+skin(?:\s*tone)?)?)\s+
           (?:community|group|background|population)\b
    """, RXFLAGS),
    "community_darker": re.compile(r"""
        \b(?:darker(?:\s+skin(?:\s*tone)?)?|deep(?:er)?(?:\s+skin(?:\s*tone)?)?)\s+
           (?:community|group|background|population)\b
    """, RXFLAGS),

    # Identity-related persona framings (linguistic self-references only)
    "persona_white": re.compile(r"""
        \b(?:white|caucasian)\b
        (?:\s+(?:community|group|background|population|skin\s*tone))?
    """, RXFLAGS),

    "persona_black": re.compile(r"""
        \bblack\b
        (?:\s+(?:community|group|background|population|skin\s*tone))?
        (?!\s*and\s*white)
    """, RXFLAGS),

    "persona_africanamerican": re.compile(r"""
        \bafrican[-\s]?american(?:s)?\b
        (?:\s+(?:community|group|background|population|skin\s*tone))?
    """, RXFLAGS),

    "persona_hispanic": re.compile(r"""
        \b(?:hispanic|latino|latina|latinx|latine)\b
        (?:\s+(?:community|group|background|population|skin\s*tone))?
    """, RXFLAGS),

    "persona_indigenous": re.compile(r"""
        \b(?:indigenous|native\s+(?:american|peoples?)|first\s+nations?|aboriginal)\b
        (?:\s+(?:community|group|background|population|skin\s*tone))?
    """, RXFLAGS),

    "persona_asian": re.compile(r"""
        \b(?:asian|east\s+asian|south\s+asian|southeast\s+asian|asian[-\s]?american)\b
        (?:\s+(?:community|group|background|population|skin\s*tone))?
    """, RXFLAGS),

    # Colorism / bias-awareness framings
    "persona_colorism": re.compile(r"""
        \b(?:colorism|skin\s*tone\s*bias|shade\s*bias|fairness\s*bias)\b
    """, RXFLAGS),
}


def add_theme_flags(df):
    txt = df['rationale'].str.lower()
    df['has_disclaimer'] = txt.str.contains(DISCLAIMER, regex=True)
    df['hedge_count']    = txt.str.count(HEDGES)
    df['normative_count']= txt.str.count(NORMATIVE)
    # Per-persona counts
    composite = 0
    for key, rx in PERSONA_PATTERNS.items():
        col = f'persona_ref__{key}'
        df[col] = txt.str.count(rx)
        composite = composite + df[col]

    # Back-compatible composite (use this where you previously used persona_refs)
    df['persona_refs'] = composite

    #df['persona_refs']   = txt.str.count(PERSONA_WORDS)
    return df

# -------------------------
# 3) Sentiment (VADER)
# -------------------------
_analyzer = SentimentIntensityAnalyzer()
def _vader(text): 
    return _analyzer.polarity_scores(text)['compound']

def add_sentiment(df):
    df['sentiment'] = df['rationale'].apply(_vader)
    return df

# -------------------------
# 4) Lengths & normalized rates
# -------------------------
def add_lengths_and_rates(df):
    df['n_words'] = df['rationale'].str.split().str.len().fillna(0).astype(int)
    # Per-100-word rates (avoid div/0)
    denom = df['n_words'].replace(0, pd.NA)
    df['hedges_per_100']    = (df['hedge_count']    / denom) * 100
    df['normative_per_100'] = (df['normative_count'] / denom) * 100
    # Per-persona rates 
    for key in PERSONA_PATTERNS:
        df[f'persona_ref_rate__{key}'] = (df[f'persona_ref__{key}'] / denom) * 100
        # composite rate (matches old behavior)
    #df['persona_per_100'] = (df['persona_refs'] / denom) * 100
    df['persona_per_100']   = (df['persona_refs']    / denom) * 100
    return df

# -------------------------
# 5) Group summaries for Results
# -------------------------
def summarize_by_group(df):
    g = df.groupby(['model','condition'], dropna=False)
    out = g.agg(
        n=('rationale','count'),
        mean_words=('n_words','mean'),
        mean_sent=('sentiment','mean'),
        pct_disclaimer=('has_disclaimer','mean'),
        mean_hedges=('hedge_count','mean'),
        mean_normative=('normative_count','mean'),
        mean_persona_refs=('persona_refs','mean'),
        hedges_per_100=('hedges_per_100','mean'),
        normative_per_100=('normative_per_100','mean'),
        persona_per_100=('persona_per_100','mean'),
    ).reset_index()

    # Formatting
    out['pct_disclaimer'] = (out['pct_disclaimer']*100).round(1)
    for c in ['mean_words','mean_sent','mean_hedges','mean_normative','mean_persona_refs',
              'hedges_per_100','normative_per_100','persona_per_100']:
        out[c] = out[c].astype(float).round(3)
    return out.sort_values(['model','condition']).reset_index(drop=True)

# -------------------------
# 6) Quote pulls (exemplars)
# -------------------------
def exemplar_quotes(df, model, condition, k=2, sort_by='hedge_count', largest=True):
    """
    Return up to k exemplar rationales for a given model & condition.
    sort_by: 'hedge_count', 'sentiment', 'n_words', or any column you added.
    largest=True -> highest values; False -> lowest.
    """
    g = df[(df['model']==model) & (df['condition']==condition)].copy()
    if g.empty:
        return []
    g = g.sort_values(sort_by, ascending=not largest)
    # Return short tuples: (image_name, rating, sort_value, rationale_snippet)
    exemplars = []
    for _, row in g.head(k).iterrows():
        snippet = re.sub(r'\s+', ' ', row['rationale']).strip()
        exemplars.append({
            'image_name': row['image_name'],
            'rating': row['rating'],
            sort_by: row[sort_by],
            'rationale': snippet
        })
    return exemplars

def exemplar_pair(df, model, condition, k_each=1, sort_by='hedge_count'):
    """
    Get a 'high vs low' pair on a chosen metric for one model/persona.
    """
    hi = exemplar_quotes(df, model, condition, k=k_each, sort_by=sort_by, largest=True)
    lo = exemplar_quotes(df, model, condition, k=k_each, sort_by=sort_by, largest=False)
    return {'high': hi, 'low': lo}

# -------------------------
# 7) Typical call sequence
# -------------------------
df_r = get_rationales(long_df)
df_r = add_theme_flags(df_r)
df_r = add_sentiment(df_r)
df_r = add_lengths_and_rates(df_r)
summary_df = summarize_by_group(df_r)
print(summary_df.head(60))

# Save summary to CSV
summary_path = os.path.join(output_path, 'rationale_summary_by_model_condition.csv')
summary_df.to_csv(summary_path, index=False)

#Example: pull quotes for GPT under 'cultural_darker'
ex = exemplar_pair(df_r, model='GPT', condition='cultural_darker', k_each=1, sort_by='hedge_count')
print("High-hedge example:", ex['high'])
print("Low-hedge example:", ex['low'])


In [ ]:

# Copy your summary_df first
tbl = summary_df.copy()

# Optional: nicer y tick labels
pretty_map = {
    'baseline':'Baseline',
    'cultural_africanamerican':'African American',
    'cultural_asian':'Asian',
    'cultural_black':'Black',
    'cultural_colorism':'Colorism',
    'cultural_darker':'Darker Community',
    'cultural_hispanic':'Hispanic',
    'cultural_indigenous':'Indigenous',
    'cultural_lighter':'Lighter Community',
    'cultural_white':'White',
    'embrace_world':'Embrace world knowledge',
    'ignore_world':'Ignore world knowledge'
}

# Fix minor typos / standardize condition names
fix = {'cultural_asain':'cultural_asian'}
tbl['condition'] = tbl['condition'].replace(fix)

# optional: order models/conditions for consistent plots
model_order = ['Claude','GPT','Gemini','Llama']
cond_order = ['baseline','cultural_africanamerican','cultural_asian','cultural_black',
              'cultural_colorism','cultural_darker','cultural_hispanic','cultural_indigenous',
              'cultural_lighter','cultural_white','embrace_world','ignore_world']
tbl['model'] = pd.Categorical(tbl['model'], categories=model_order, ordered=True)
tbl['condition'] = pd.Categorical(tbl['condition'], categories=cond_order, ordered=True)

# keep only the metrics you want in the paper (we’ve removed normative language)
metrics = ['mean_words', 'hedges_per_100', 'persona_per_100', 'mean_sent'] 

# weight by n (images per condition) so conditions contribute proportionally
w = tbl.assign(w=tbl['n'].clip(lower=1))

per_model = (
    tbl.assign(
        words_w    = tbl['mean_words']      * w['w'],
        hedges_w   = tbl['hedges_per_100']  * w['w'],
        persona_w  = tbl['persona_per_100'] * w['w'],
        sent_w     = tbl['mean_sent']       * w['w']
    )
    .groupby('model', as_index=False)
    .agg(
        n_total=('n','sum'),
        mean_words=('words_w','sum'),
        hedges_per_100=('hedges_w','sum'),
        persona_per_100=('persona_w','sum'),
        mean_sent=('sent_w','sum'),
    )
)

per_model['mean_words']       = (per_model['mean_words']       / per_model['n_total']).round(2)
per_model['hedges_per_100']   = (per_model['hedges_per_100']   / per_model['n_total']).round(3)
per_model['persona_per_100']  = (per_model['persona_per_100']  / per_model['n_total']).round(3)
per_model['mean_sent']        = (per_model['mean_sent']        / per_model['n_total']).round(3)

print(per_model.to_string(index=False))

# Export for paper
path = os.path.join(output_path, 'per_model_qual_summary.csv')
per_model.to_csv(path, index=False)
with open('per_model_qual_summary.tex','w') as f:
    f.write(per_model.to_latex(index=False, caption='Per-model qualitative rationale summary',
                               label='tab:per_model_qual', float_format="%.3f"))

fig, ax = plt.subplots(figsize=(7,5))
for m in model_order:
    d = tbl[tbl['model']==m]
    ax.scatter(d['mean_words'], d['hedges_per_100'],
               s=20 + 80*(d['persona_per_100'] / (1 + d['persona_per_100']).clip(upper=3)),
               label=m, alpha=0.9)

ax.set_xlabel('Rationale length (mean words)')
ax.set_ylabel('Hedges per 100 words')
ax.set_title('Rationale style by model and condition')
ax.legend(title='Model', frameon=False)
ax.grid(True, alpha=0.2)
plt.tight_layout()
plt.savefig(output_path + 'style_scatter_length_vs_hedges.png', dpi=300)
plt.show()

paper_table = per_model[['model','mean_words','hedges_per_100','persona_per_100','mean_sent']].copy()
paper_table.columns = ['Model','Mean words','Hedges/100','Persona refs/100','Mean sentiment']
print(paper_table.to_string(index=False))

paper_table.to_csv('paper_table_qualitative_summary.csv', index=False)
with open('paper_table_qualitative_summary.tex','w') as f:
    f.write(paper_table.to_latex(index=False, caption='Qualitative rationale metrics by model',
                                 label='tab:qual_model_summary', float_format="%.3f"))
    
def lollipop_axis(ax, d, metric, title):
    d = d.sort_values('condition')
    y = np.arange(len(d))
    ax.hlines(y, 0, d[metric], linewidth=1)
    ax.plot(d[metric], y, 'o')
    ax.set_yticks(y)
    ax.set_yticklabels([pretty_map.get(c, str(c)) for c in d['condition']])
    ax.set_xlabel(metric.replace('_',' '))
    ax.set_title(title, weight='bold')
    ax.grid(True, axis='x', alpha=0.2)

metric = 'hedges_per_100'
fig, axes = plt.subplots(2, 2, figsize=(12, 7), sharex=True)
axes = axes.ravel()

# Determine a common x-limit across all models for comparability
xmin = 0
xmax = float(tbl[metric].max()) * 1.1  # small headroom

for i, m in enumerate(model_order):
    d = tbl[tbl['model']==m]
    lollipop_axis(axes[i], d, metric, f'Hedges/100 — {m}')
    axes[i].set_xlim(xmin, xmax)

# Panel labels A–D (optional)
panel_labels = ['A','B','C','D']
for ax, lab in zip(axes, panel_labels):
    ax.text(0.01, 1.1, lab, transform=ax.transAxes, va='top', ha='left', fontsize=12, weight='bold')

fig.suptitle('Hedging by persona (lollipop plots per model)', y=0.995, weight='bold')
fig.tight_layout(rect=[0,0,1,0.96])
plt.savefig('figure_hedges_all_models.png', dpi=300)
plt.show()

metric = 'persona_per_100'
fig, axes = plt.subplots(2, 2, figsize=(12, 7), sharex=True)
axes = axes.ravel()

xmin = 0
xmax = float(tbl[metric].max()) * 1.1  # common x-limit

for i, m in enumerate(model_order):
    d = tbl[tbl['model']==m]
    lollipop_axis(axes[i], d, metric, f'Persona refs/100 — {m}')
    axes[i].set_xlim(xmin, xmax)

panel_labels = ['A','B','C','D']
for ax, lab in zip(axes, panel_labels):
    ax.text(0.01, 1.1, lab, transform=ax.transAxes, va='top', ha='left', fontsize=12, weight='bold')

fig.suptitle('References to identity constructs for each prompt', y=0.995, weight='bold')
fig.tight_layout(rect=[0,0,1,0.96])
plt.savefig('figure_persona_refs_all_models.png', dpi=300)
plt.show()

# --- Sentiment lollipop plots by model/persona (VADER compound) ---

metric = 'mean_sent'  # already in summary_df as produced earlier
fig, axes = plt.subplots(2, 2, figsize=(12, 7), sharex=True)
axes = axes.ravel()

# VADER compound ranges from -1 to 1. Keep a symmetric axis for comparability.
xmin, xmax = -1.0, 1.0

for i, m in enumerate(model_order):
    d = tbl[tbl['model'] == m].copy()
    # Safety: ensure numeric
    d[metric] = pd.to_numeric(d[metric], errors='coerce')
    lollipop_axis(axes[i], d, metric, f'{m}')
    axes[i].set_xlim(xmin, xmax)
    # Zero-line for polarity neutrality
    axes[i].axvline(0, linestyle='--', linewidth=0.8, alpha=0.6)
    # Add x-axis label
    axes[i].set_xlabel('VADER compound sentiment score')

# Panel labels A–D
panel_labels = ['A','B','C','D']
for ax, lab in zip(axes, panel_labels):
    ax.text(0.01, 1.1, lab, transform=ax.transAxes, va='top', ha='left', fontsize=12, weight='bold')

fig.suptitle('Valence (VADER) by persona for each model', y=0.995, weight='bold')
fig.tight_layout(rect=[0,0,1,0.96])
plt.savefig(os.path.join(output_path, 'figure_valence_all_models.png'), dpi=300)
plt.show()



In [ ]:
# --- add colors for models ---
model_colors = {
    'GPT': '#2ecc71',    # Green
    'Claude': '#e74c3c', # Red
    'Llama': '#3498db',  # Blue
    'Gemini': '#f1c40f', # Yellow
}
# fallback palette if extra models exist
default_palette = ['#8e44ad', '#16a085', '#d35400', '#2c3e50', '#7f8c8d']
from itertools import cycle
_default_cycle = cycle(default_palette)

models_present = [m for m in model_order if m in tbl['model'].unique()]
model_to_color = {}
for m in models_present:
    # try exact family match, else fallback to cycle
    family = next((f for f in model_colors if f.lower() in str(m).lower()), None)
    model_to_color[m] = model_colors.get(family, next(_default_cycle))

# Modified lollipop_axis to take color
def lollipop_axis(ax, d, metric, title, color='#333333'):
    d = d.sort_values('condition')
    y = np.arange(len(d))
    # horizontal lollipop stems
    ax.hlines(y, 0, d[metric], linewidth=1.5, color=color, alpha=0.9)
    # points: filled circle with thin black edge for legibility
    ax.plot(d[metric], y, 'o', markersize=7, markerfacecolor=color, markeredgecolor='k', markeredgewidth=0.4)
    ax.set_yticks(y)
    ax.set_yticklabels([pretty_map.get(c, str(c)) for c in d['condition']])
    ax.set_xlabel(metric.replace('_',' '))
    ax.set_title(title, weight='bold')
    ax.grid(True, axis='x', alpha=0.2)

# Common variables used in three blocks
metric = 'hedges_per_100'
fig, axes = plt.subplots(2, 2, figsize=(12, 7), sharex=True)
axes = axes.ravel()

# Determine a common x-limit across all models for comparability
xmin = 0
xmax = float(tbl[metric].max()) * 1.1  # small headroom

for i, m in enumerate(model_order):
    d = tbl[tbl['model']==m]
    color = model_to_color.get(m, '#444444')
    lollipop_axis(axes[i], d, metric, f'Hedges/100 — {m}', color=color)
    axes[i].set_xlim(xmin, xmax)

# Panel labels A–D (optional)
panel_labels = ['A','B','C','D']
for ax, lab in zip(axes, panel_labels):
    ax.text(0.01, 1.1, lab, transform=ax.transAxes, va='top', ha='left', fontsize=12, weight='bold')

fig.suptitle('Hedging by persona (lollipop plots per model)', y=0.995, weight='bold')
fig.tight_layout(rect=[0,0,1,0.96])
plt.savefig('figure_hedges_all_models_colored.png', dpi=300)
plt.show()

# --- Persona refs per 100 ---
metric = 'persona_per_100'
fig, axes = plt.subplots(2, 2, figsize=(12, 7), sharex=True)
axes = axes.ravel()

xmin = 0
xmax = float(tbl[metric].max()) * 1.1  # common x-limit

for i, m in enumerate(model_order):
    d = tbl[tbl['model']==m]
    color = model_to_color.get(m, '#444444')
    lollipop_axis(axes[i], d, metric, f'Persona refs/100 — {m}', color=color)
    axes[i].set_xlim(xmin, xmax)

panel_labels = ['A','B','C','D']
for ax, lab in zip(axes, panel_labels):
    ax.text(0.01, 1.1, lab, transform=ax.transAxes, va='top', ha='left', fontsize=12, weight='bold')

fig.suptitle('References to identity constructs for each prompt', y=0.995, weight='bold')
fig.tight_layout(rect=[0,0,1,0.96])
plt.savefig('figure_persona_refs_all_models_colored.png', dpi=300)
plt.show()

# --- Sentiment lollipop plots by model/persona (VADER compound) ---
metric = 'mean_sent'  # already in summary_df as produced earlier
fig, axes = plt.subplots(2, 2, figsize=(12, 7), sharex=True)
axes = axes.ravel()

# VADER compound ranges from -1 to 1. Keep a symmetric axis for comparability.
xmin, xmax = -1.0, 1.0

for i, m in enumerate(model_order):
    d = tbl[tbl['model'] == m].copy()
    # Safety: ensure numeric
    d[metric] = pd.to_numeric(d[metric], errors='coerce')
    color = model_to_color.get(m, '#444444')
    lollipop_axis(axes[i], d, metric, f'{m}', color=color)
    axes[i].set_xlim(xmin, xmax)
    # Zero-line for polarity neutrality
    axes[i].axvline(0, linestyle='--', linewidth=0.8, alpha=0.6)
    # Add x-axis label
    axes[i].set_xlabel('VADER compound sentiment score')

# Panel labels A–D
panel_labels = ['A','B','C','D']
for ax, lab in zip(axes, panel_labels):
    ax.text(0.01, 1.1, lab, transform=ax.transAxes, va='top', ha='left', fontsize=12, weight='bold')

fig.suptitle('Valence (VADER) by persona for each model', y=0.995, weight='bold')
fig.tight_layout(rect=[0,0,1,0.96])
plt.savefig(os.path.join(output_path, 'figure_valence_all_models_colored.png'), dpi=300)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.multitest import multipletests

# ---------- 1) Compute within-model tests vs baseline ----------
def stars_from_p(p):
    if pd.isna(p):
        return ""
    return "****" if p < 1e-4 else ("***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "")))

def sentiment_tests_vs_baseline(df_r, model_order, baseline="baseline"):
    """
    df_r must have columns: model, condition, sentiment
    Returns dict: star_map[model][condition] = '***' ('' for baseline or ns)
    """
    star_map = {m: {} for m in model_order}
    for m in model_order:
        d = df_r[df_r['model'] == m].copy()
        # Per-rationale sentiment vectors
        base = d[d['condition'] == baseline]['sentiment'].dropna().values
        if base.size == 0:
            # No baseline data; skip stars for this model
            continue

        # Gather p-values for all non-baseline conditions
        conds = sorted(c for c in d['condition'].unique() if c != baseline)
        pvals = []
        kept_conds = []
        for c in conds:
            x = d[d['condition'] == c]['sentiment'].dropna().values
            if x.size == 0:
                pvals.append(np.nan); kept_conds.append(c); continue
            # Welch t-test (unequal variances)
            t, p = stats.ttest_ind(x, base, equal_var=False, nan_policy='omit')
            pvals.append(p); kept_conds.append(c)

        # Holm adjust within model
        pvals_arr = np.array([np.nan if pd.isna(p) else p for p in pvals], dtype=float)
        # mask nans for adjustment
        valid_mask = ~np.isnan(pvals_arr)
        adj = np.full_like(pvals_arr, np.nan)
        if valid_mask.sum() > 0:
            _, p_adj, _, _ = multipletests(pvals_arr[valid_mask], method='holm')
            adj[valid_mask] = p_adj

        # Fill stars
        for c, p_adj in zip(kept_conds, adj):
            star_map[m][c] = stars_from_p(p_adj)
        # Baseline never has a star
        star_map[m][baseline] = ""
    return star_map

# Build star map from your per-rationale DF
# df_r came from your earlier pipeline (get_rationales -> add_theme_flags -> add_sentiment, etc.)
star_map = sentiment_tests_vs_baseline(df_r, model_order, baseline="baseline")

# ---------- 2) Plot sentiment lollipop + stars ----------
metric = 'mean_sent'
fig, axes = plt.subplots(2, 2, figsize=(12, 7), sharex=True)
axes = axes.ravel()
xmin, xmax = -1.0, 1.0  # VADER compound range; keep symmetric

for i, m in enumerate(model_order):
    d = tbl[tbl['model'] == m].copy()  # persona-level means from your summary
    d[metric] = pd.to_numeric(d[metric], errors='coerce')
    lollipop_axis(axes[i], d, metric, f'Sentiment (VADER) — {m}')
    axes[i].set_xlim(xmin, xmax)
    axes[i].axvline(0, linestyle='--', linewidth=0.8, alpha=0.6)

    # --- annotate stars next to each persona (except baseline) ---
    # Recreate the y positions used inside lollipop_axis
    d = d.sort_values('condition')
    y = np.arange(len(d))
    # small horizontal offset for the star placement
    xpad = 0.02 * (xmax - xmin)

    for yy, (_, row) in zip(y, d.iterrows()):
        cond = row['condition']
        if cond == 'baseline':
            continue
        xval = row[metric]
        star = star_map.get(m, {}).get(cond, "")
        if not star:
            continue
        # If near the right edge, place stars to the left
        if pd.notna(xval):
            xp = xval + xpad if (xval + xpad) < (xmax - 0.02) else xval - xpad
            ha = 'left' if xp >= xval else 'right'
            axes[i].text(xp, yy, star, va='center', ha=ha, fontsize=11, weight='bold')

# Panel labels
for ax, lab in zip(axes, ['A','B','C','D']):
    ax.text(0.01, 1.1, lab, transform=ax.transAxes, va='top', ha='left', fontsize=12, weight='bold')

fig.suptitle('Sentiment by persona with significance vs baseline (Holm-adjusted)', y=0.995, weight='bold')
fig.tight_layout(rect=[0,0,1,0.96])
plt.savefig(os.path.join(output_path, 'figure_sentiment_all_models_with_stars.png'), dpi=300)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests

df_r = get_rationales(long_df)
df_r = add_sentiment(df_r)

def stars_from_p(p):
    if pd.isna(p):
        return ""
    return "****" if p < 1e-4 else ("***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "")))

def sentiment_tests_vs_baseline(df_r, model_order, baseline="baseline"):
    star_map = {m: {} for m in model_order}
    results = []
    for m in model_order:
        d = df_r[df_r['model'] == m].copy()
        base = d[d['condition'] == baseline]['sentiment'].dropna().values
        if base.size == 0:
            continue
        conds = sorted(c for c in d['condition'].unique() if c != baseline)
        pvals = []
        kept_conds = []
        for c in conds:
            x = d[d['condition'] == c]['sentiment'].dropna().values
            if x.size == 0:
                pvals.append(np.nan); kept_conds.append(c); continue
            t, p = stats.ttest_ind(x, base, equal_var=False, nan_policy='omit')
            pvals.append(p); kept_conds.append(c)
        pvals_arr = np.array([np.nan if pd.isna(p) else p for p in pvals], dtype=float)
        valid_mask = ~np.isnan(pvals_arr)
        adj = np.full_like(pvals_arr, np.nan)
        if valid_mask.sum() > 0:
            _, p_adj, _, _ = multipletests(pvals_arr[valid_mask], method='holm')
            adj[valid_mask] = p_adj
        for c, raw, padj in zip(kept_conds, pvals, adj):
            star = stars_from_p(padj)
            star_map[m][c] = star
            results.append({
                "model": m,
                "condition": c,
                "p_raw": raw,
                "p_adj": padj,
                "stars": star
            })
    return pd.DataFrame(results)

model_order = ['Claude','GPT','Gemini','Llama']
results_df = sentiment_tests_vs_baseline(df_r, model_order)
results_df


In [ ]:
# 0) Normalize spelling in BOTH dataframes
fix = {'cultural_asain': 'cultural_asian'}
results_df['condition'] = results_df['condition'].replace(fix)
tbl['condition']        = tbl['condition'].replace(fix)

# 1) Keep only significant results and build the lookup
sig_results = results_df[results_df['p_adj'] < 0.05].copy()
star_map = { (row['model'], row['condition']) : row['stars']
             for _, row in sig_results.iterrows() }

print("Significant (model, condition) pairs:", list(star_map.keys()))
# You should see: [('Claude','cultural_asian')] based on your table

# 2) Plot with correct condition lookup + robust y positions
metric = 'mean_sent'
fig, axes = plt.subplots(2, 2, figsize=(12, 7), sharex=True)
axes = axes.ravel()
xmin, xmax = -1.0, 1.0  # symmetric for VADER

for i, m in enumerate(model_order):
    d = tbl[tbl['model'] == m].copy()
    d = d.sort_values('condition')                # match lollipop_axis y-order
    d[metric] = pd.to_numeric(d[metric], errors='coerce')

    # draw the lollipop panel
    lollipop_axis(axes[i], d, metric, f'Sentiment (VADER) — {m}')
    axes[i].set_xlim(xmin, xmax)
    axes[i].axvline(0, linestyle='--', linewidth=0.8, alpha=0.6)

    # exact y positions used by lollipop_axis
    y_positions = np.arange(len(d))
    # small horizontal offset so stars don't overlap the dot
    xpad = 0.04 * (xmax - xmin)

    # annotate using the ACTUAL condition string from the row
    for yy, (_, row) in enumerate(d.iterrows()):
        cond = row['condition']
        xval = row[metric]
        star = star_map.get((m, cond), "")
        if star and pd.notna(xval):
            # place star to right; if near right edge, flip to left
            xp  = xval + xpad if (xval + xpad) < (xmax - 0.02) else xval - xpad
            hal = 'left' if xp >= xval else 'right'
            axes[i].text(xp, yy, star,
                         va='center', ha=hal,
                         fontsize=14, weight='bold',
                         color='black', zorder=10)

# panel labels
for ax, lab in zip(axes, ['A','B','C','D']):
    ax.text(0.01, 1.1, lab, transform=ax.transAxes,
            va='top', ha='left', fontsize=12, weight='bold')

fig.suptitle('Sentiment by persona with significance vs baseline (Holm-adjusted)',
             y=0.995, weight='bold')
fig.tight_layout(rect=[0,0,1,0.96])
plt.savefig(os.path.join(output_path, 'figure_sentiment_all_models_with_stars.png'), dpi=300)
plt.show()


In [ ]:
# Hedging significance tests vs baseline
df_r = get_rationales(long_df)
df_r = add_theme_flags(df_r)  # need hedge_count
df_r = add_lengths_and_rates(df_r)  # need hedges_per_100

def stars_from_p(p):
    if pd.isna(p):
        return ""
    return "****" if p < 1e-4 else ("***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "")))

def hedging_tests_vs_baseline(df_rr, model_order, baseline='baseline', min_n=5):
    """
    Welch t-tests comparing hedges_per_100 between each persona and baseline within each model.
    Holm correction is applied across personas per model.
    min_n: require at least this many rationales per group to test; else p=nan.
    Returns: results_df and star_map usable for plotting.
    """
    out = []
    star_map = {m: {} for m in model_order}
    for m in model_order:
        d = df_rr[(df_rr['model'] == m)].copy()
        base = d[d['condition'] == baseline]['hedges_per_100'].dropna()
        if base.shape[0] < min_n:
            continue
        conds = [c for c in d['condition'].dropna().unique() if c != baseline]
        pvals, kept = [], []
        for c in conds:
            x = d[d['condition'] == c]['hedges_per_100'].dropna()
            if x.shape[0] < min_n:
                pvals.append(np.nan); kept.append(c); continue
            t, p = stats.ttest_ind(x, base, equal_var=False, nan_policy='omit')
            pvals.append(p); kept.append(c)
        pvals = np.array(pvals, dtype=float)
        mask = ~np.isnan(pvals)
        p_adj = np.full_like(pvals, np.nan)
        if mask.sum() > 0:
            _, p_adj[mask], _, _ = multipletests(pvals[mask], method='holm')
        for c, p_raw, p_a in zip(kept, pvals, p_adj):
            out.append({"model": m, "condition": c, "p_raw": p_raw, "p_adj": p_a, "stars": stars_from_p(p_a)})
            star_map[m][c] = stars_from_p(p_a)
        star_map[m][baseline] = ""
    results_df = pd.DataFrame(out).sort_values(['model','condition']).reset_index(drop=True)
    return results_df, star_map

# Run tests
model_order = ['Claude','GPT','Gemini','Llama']
hedge_results_df, hedge_star_map_by_model = hedging_tests_vs_baseline(df_r, model_order, baseline='baseline', min_n=5)

# (Optional) quick look
print(hedge_results_df.to_string(index=False))

# Keep plotting DF in sync with typo fix and build a flat star lookup
tbl['condition'] = tbl['condition'].replace(fix)

# Flatten star map for convenient lookup
hedge_star_map = {(m, c): s for m, d in hedge_star_map_by_model.items() for c, s in d.items()}

metric = 'hedges_per_100'
fig, axes = plt.subplots(2, 2, figsize=(12, 7), sharex=True)
axes = axes.ravel()

xmin = 0
xmax = float(tbl[metric].max()) * 1.1 if np.isfinite(tbl[metric].max()) else 1.0

for i, m in enumerate(model_order):
    d = tbl[tbl['model'] == m].copy().sort_values('condition')
    lollipop_axis(axes[i], d, metric, f'Hedges/100 — {m}')
    axes[i].set_xlim(xmin, xmax)

    # annotate stars next to points
    y_positions = np.arange(len(d))
    xpad = 0.02 * (xmax - xmin) if (xmax - xmin) > 0 else 0.05
    for yy, (_, row) in enumerate(d.iterrows()):
        cond = row['condition']
        xval = row[metric]
        star = hedge_star_map.get((m, cond), "")
        if star and pd.notna(xval):
            xp  = xval + xpad if (xval + xpad) < (xmax - 0.02) else xval - xpad
            hal = 'left' if xp >= xval else 'right'
            axes[i].text(xp, yy, star, va='center', ha=hal,
                         fontsize=14, weight='bold', color='black', zorder=10)

# panel labels
for ax, lab in zip(axes, ['A','B','C','D']):
    ax.text(0.01, 1.1, lab, transform=ax.transAxes, va='top', ha='left', fontsize=12, weight='bold')

fig.suptitle('Hedging by persona with significance vs baseline (Holm-adjusted)', y=0.995, weight='bold')
fig.tight_layout(rect=[0,0,1,0.96])
plt.savefig(os.path.join(output_path, 'figure_hedges_all_models_with_stars.png'), dpi=300)
plt.show()


## Combinging Quant and Qual Results

In [ ]:
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
from scipy import stats
SHORT = {
    'baseline':'BL', 'ignore_world':'IW', 'embrace_world':'EW',
    'cultural_white':'WH', 'cultural_black':'BK', 'cultural_asian':'AS',
    'cultural_hispanic':'HI', 'cultural_indigenous':'IN',
    'cultural_africanamerican':'AA', 'cultural_colorism':'CL',
    'cultural_lighter':'LG', 'cultural_darker':'DK'
}
import matplotlib.patheffects as pe
import numpy as np
import pandas as pd

def annotate_points(ax, d, xcol='delta_emm', ycol='delta_hedges',
                    label_col='condition', emm_star_col='emm_stars',
                    hedges_sig_lookup=None, topk=3):
    """
    Labels significant points; if none, labels top-K by |x|+|y|.
    hedges_sig_lookup: dict {(model, condition) -> stars or ''}.
    """
    # which are "significant" by either channel?
    sig_mask = d[emm_star_col].astype(str).str.len().gt(0)
    if hedges_sig_lookup is not None:
        sig_mask = sig_mask | d.apply(lambda r: bool(hedges_sig_lookup.get((str(r['model']), str(r['condition'])), "")), axis=1)

    lab_df = d[sig_mask].copy()
    if lab_df.empty:
        # fall back to top-K biggest shifts
        d2 = d.assign(mag=(d[xcol].abs() + d[ycol].abs()))
        lab_df = d2.sort_values('mag', ascending=False).head(topk)

    # simple de-overlap: jitter labels that sit too close
    used = []
    for _, row in lab_df.iterrows():
        x, y = row[xcol], row[ycol]
        # offset label a bit away from origin to reduce dot overlap
        dx = 0.02 * (np.sign(x) if x != 0 else 1)
        dy = 0.02 * (np.sign(y) if y != 0 else 1)
        tx, ty = x + dx, y + dy

        # nudge if colliding with previous labels
        for _ in range(10):
            if any(abs(tx-ux) < 0.03 and abs(ty-uy) < 0.03 for (ux, uy) in used):
                tx += 0.02
                ty += 0.02
            else:
                break
        used.append((tx, ty))

        lab = SHORT.get(str(row[label_col]), str(row[label_col]))
        ax.text(tx, ty, lab,
                ha='left', va='center', fontsize=9, weight='bold',
                path_effects=[pe.withStroke(linewidth=3, foreground='white')])

        # thin leader line from label to point
        ax.plot([x, tx], [y, ty], linewidth=0.6, alpha=0.6)

    # legend key for abbreviations (optional: include in caption instead)




emm_path = "outputs/cross_model_emms.csv"  # <-- set this to your file path
emm = pd.read_csv(emm_path)

# -----------------------------
# 1) Standardize condition keys
# -----------------------------
fix = {'cultural_asain': 'cultural_asian'}
for d in (emm, summary_df):
    d['condition'] = d['condition'].replace(fix)

model_order = ['Claude','GPT','Gemini','Llama']
cond_order = ['baseline','cultural_africanamerican','cultural_asian','cultural_black',
              'cultural_colorism','cultural_darker','cultural_hispanic','cultural_indigenous',
              'cultural_lighter','cultural_white','embrace_world','ignore_world']

# -----------------------------
# 2) ΔEMM vs baseline within model
#    (uses SE combination as an approximation if you don't have contrast SEs)
# -----------------------------
# Baseline per model
base_emm = emm[emm['condition']=='baseline'].set_index('model')[['emmean','SE']].rename(
    columns={'emmean':'base_emm','SE':'base_SE'}
)

emm2 = emm.merge(base_emm, left_on='model', right_index=True, how='left')
emm2['delta_emm'] = emm2['emmean'] - emm2['base_emm']

# Approx SE for the difference (assumes independence; conservative if cov>0)
emm2['SE_delta'] = np.sqrt(emm2['SE']**2 + emm2['base_SE']**2)

# 95% CI
zcrit = stats.norm.ppf(0.975)
emm2['delta_emm_lo'] = emm2['delta_emm'] - zcrit*emm2['SE_delta']
emm2['delta_emm_hi'] = emm2['delta_emm'] + zcrit*emm2['SE_delta']

# Per-model Holm-adjusted p-values vs baseline (two-sided z-tests)
def holm_stars(group):
    # exclude baseline itself
    g = group[group['condition']!='baseline'].copy()
    # some rows could have SE_delta == 0 if degenerate; guard:
    g['z'] = g['delta_emm'] / g['SE_delta'].replace(0, np.nan)
    g['p_raw'] = 2*(1-stats.norm.cdf(np.abs(g['z'])))
    if g['p_raw'].notna().any():
        mask = g['p_raw'].notna().values
        p_adj = np.full(len(g), np.nan)
        p_adj[mask] = multipletests(g.loc[mask, 'p_raw'], method='holm')[1]
        g['p_adj'] = p_adj
    else:
        g['p_adj'] = np.nan
    # star mapping
    def stars(p):
        if pd.isna(p): return ""
        return "****" if p < 1e-4 else ("***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "")))
    g['emm_stars'] = g['p_adj'].apply(stars)
    return g[['model','condition','delta_emm','delta_emm_lo','delta_emm_hi','SE_delta','p_adj','emm_stars']]

emm_delta = (emm2.groupby('model', group_keys=False).apply(holm_stars)
             .reset_index(drop=True))

# Include baseline row (Δ=0, no stars) so plotting can show all personas aligned
baseline_rows = emm2[emm2['condition']=='baseline'][['model','condition']].copy()
baseline_rows['delta_emm'] = 0.0
baseline_rows['delta_emm_lo'] = 0.0
baseline_rows['delta_emm_hi'] = 0.0
baseline_rows['SE_delta'] = np.nan
baseline_rows['p_adj'] = np.nan
baseline_rows['emm_stars'] = ""

emm_delta = pd.concat([emm_delta, baseline_rows], ignore_index=True)

# -----------------------------
# 3) ΔHedges/100 vs baseline within model
# -----------------------------
tbl = summary_df.copy()
base_h = tbl[tbl['condition']=='baseline'][['model','hedges_per_100']].rename(columns={'hedges_per_100':'base_h'})
tbl = tbl.merge(base_h, on='model', how='left')
tbl['delta_hedges'] = tbl['hedges_per_100'] - tbl['base_h']

# Optional: pull Holm-adjusted significance for hedges (if you have hedge_results_df)
# fallback: no hedges stars if not available
hedges_sig = {}
try:
    # Expect columns: model, condition, p_adj
    tmp = hedge_results_df.copy()
    tmp['condition'] = tmp['condition'].replace(fix)
    tmp['hedge_stars'] = tmp['p_adj'].apply(lambda p:
        ("****" if p < 1e-4 else ("***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "")))) if pd.notna(p) else ""
    )
    hedges_sig = {(r.model, r.condition): r.hedge_stars for r in tmp.itertuples(index=False)}
except NameError:
    # silently proceed without hedges stars
    pass

# -----------------------------
# 4) Merge Δ–Δ and add size/channel
# -----------------------------
dd = (tbl[['model','condition','delta_hedges','persona_per_100']]
      .merge(emm_delta, on=['model','condition'], how='inner'))

# Ensure consistent ordering & dtypes
dd['model'] = pd.Categorical(dd['model'], categories=model_order, ordered=True)
dd['condition'] = pd.Categorical(dd['condition'], categories=cond_order, ordered=True)
dd = dd.sort_values(['model','condition'])

# -----------------------------
# 5) Plot Δ–Δ bivariate by model (2×2 facets)
# -----------------------------
fig, axes = plt.subplots(2, 2, figsize=(12, 9), sharex=True, sharey=True)
axes = axes.ravel()

# Axis limits with some headroom
xpad = 0.05 * np.nanmax(np.abs(dd['delta_emm'].values)) if np.isfinite(np.nanmax(np.abs(dd['delta_emm'].values))) else 0.2
ypad = 0.05 * np.nanmax(np.abs(dd['delta_hedges'].values)) if np.isfinite(np.nanmax(np.abs(dd['delta_hedges'].values))) else 0.2
xmax = np.nanmax(np.abs(dd['delta_emm'].values)) + xpad
ymax = np.nanmax(np.abs(dd['delta_hedges'].values)) + ypad

for i, m in enumerate(model_order):
    ax = axes[i]
    d = dd[dd['model']==m].copy()

    # zero lines
    ax.axvline(0, linestyle='--', linewidth=0.8, alpha=0.6)
    ax.axhline(0, linestyle='--', linewidth=0.8, alpha=0.6)

    # scatter points
    sizes = 25 + 60 * (d['persona_per_100'].fillna(0) / (1 + d['persona_per_100'].fillna(0))).clip(upper=3)
    pts = ax.scatter(d['delta_emm'], d['delta_hedges'], s=sizes)

    # horizontal error bars for ΔEMM (95% CI)
    ax.errorbar(d['delta_emm'], d['delta_hedges'],
                xerr=[d['delta_emm'] - d['delta_emm_lo'], d['delta_emm_hi'] - d['delta_emm']],
                fmt='none', linewidth=1, alpha=0.7, capsize=2)

    # star for EMM significance
    for x, y, star in zip(d['delta_emm'], d['delta_hedges'], d['emm_stars']):
        if isinstance(star, str) and star != "":
            ax.text(x, y + 0.02*ymax, star, ha='center', va='bottom', fontsize=11, weight='bold')

    # bold outline for hedges significance
    for (x, y, cond) in zip(d['delta_emm'], d['delta_hedges'], d['condition'].astype(str)):
        star = hedges_sig.get((m, cond), "")
        if star != "":
            ax.scatter([x], [y], s= sizes.iloc[0] if np.ndim(sizes)==0 else 0, facecolors='none', linewidths=2)

    # label a few key personas to aid reading (optional)
    annotate_points(
    ax, d,
    xcol='delta_emm', ycol='delta_hedges',
    label_col='condition',
    emm_star_col='emm_stars',
    hedges_sig_lookup=hedges_sig  # your {(model, condition): stars} dict or None
)

    ax.set_title(m, weight='bold')
    ax.set_xlim(-xmax, xmax)
    ax.set_ylim(-ymax, ymax)
    ax.grid(True, alpha=0.2)

fig.suptitle('Persona effects by model: ΔEMM (x) vs ΔHedges/100 (y)\nStars: ΔEMM Holm-significant; Bold outline: ΔHedges Holm-significant', y=0.98, weight='bold')
for ax in axes[::2]: ax.set_ylabel('Δ Hedges per 100 words')
for ax in axes[2:]:  ax.set_xlabel('Δ EMM (persona − baseline)')

plt.tight_layout(rect=[0,0,1,0.94])
plt.savefig(os.path.join(".", "figure_deltaEMM_vs_deltaHedges_by_model.png"), dpi=300)
plt.show()


In [ ]:
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
from scipy import stats
import matplotlib.patheffects as pe

# load summary df 
summary_df = pd.read_csv(os.path.join(output_path, 'rationale_summary_by_model_condition.csv'))
# --- short labels for personas ---
SHORT = {
    'baseline':'BL', 'ignore_world':'IW', 'embrace_world':'EW',
    'cultural_white':'WH', 'cultural_black':'BK', 'cultural_asian':'AS',
    'cultural_hispanic':'HI', 'cultural_indigenous':'IN',
    'cultural_africanamerican':'AA', 'cultural_colorism':'CL',
    'cultural_lighter':'LG', 'cultural_darker':'DK'
}

def annotate_points(ax, d, xcol='delta_emm', ycol='delta_hedges',
                    label_col='condition'):
    """
    Label ALL points with persona abbreviations (no significance filtering).
    """
    used = []
    for _, row in d.iterrows():
        x, y = row[xcol], row[ycol]
        # small outward nudge so the label isn’t on top of the dot
        dx = 0.02 * (1 if x >= 0 else -1)
        dy = 0.02 * (1 if y >= 0 else -1)
        tx, ty = x + dx, y + dy

        # simple de-overlap: nudge if colliding with previous labels
        for _ in range(10):
            if any(abs(tx-ux) < 0.03 and abs(ty-uy) < 0.03 for (ux, uy) in used):
                tx += 0.02; ty += 0.02
            else:
                break
        used.append((tx, ty))

        lab = SHORT.get(str(row[label_col]), str(row[label_col]))
        ax.text(tx, ty, lab, ha='left', va='center', fontsize=9, weight='bold',
                path_effects=[pe.withStroke(linewidth=3, foreground='white')])
        # leader line
        ax.plot([x, tx], [y, ty], linewidth=0.6, alpha=0.6)

# -----------------------------
# 0) Inputs
# -----------------------------
# `summary_df` must already be in memory
emm_path = "outputs/cross_model_emms.csv"
emm = pd.read_csv(emm_path)

# -----------------------------
# 1) Standardize condition keys
# -----------------------------
fix = {'cultural_asain': 'cultural_asian'}
for d in (emm, summary_df):
    d['condition'] = d['condition'].replace(fix)

model_order = ['Claude','GPT','Gemini','Llama']
cond_order = ['baseline','cultural_africanamerican','cultural_asian','cultural_black',
              'cultural_colorism','cultural_darker','cultural_hispanic','cultural_indigenous',
              'cultural_lighter','cultural_white','embrace_world','ignore_world']

# -----------------------------
# 2) ΔEMM vs baseline within model
# -----------------------------
base_emm = (emm[emm['condition']=='baseline']
            .set_index('model')[['emmean','SE']]
            .rename(columns={'emmean':'base_emm','SE':'base_SE'}))

emm2 = emm.merge(base_emm, left_on='model', right_index=True, how='left')
emm2['delta_emm'] = emm2['emmean'] - emm2['base_emm']
emm2['SE_delta']  = np.sqrt(emm2['SE']**2 + emm2['base_SE']**2)  # conservative approx

zcrit = stats.norm.ppf(0.975)
emm2['delta_emm_lo'] = emm2['delta_emm'] - zcrit*emm2['SE_delta']
emm2['delta_emm_hi'] = emm2['delta_emm'] + zcrit*emm2['SE_delta']

def holm_stars(group):
    g = group[group['condition']!='baseline'].copy()
    g['z'] = g['delta_emm'] / g['SE_delta'].replace(0, np.nan)
    g['p_raw'] = 2*(1-stats.norm.cdf(np.abs(g['z'])))
    if g['p_raw'].notna().any():
        mask = g['p_raw'].notna().values
        p_adj = np.full(len(g), np.nan)
        p_adj[mask] = multipletests(g.loc[mask, 'p_raw'], method='holm')[1]
        g['p_adj'] = p_adj
    else:
        g['p_adj'] = np.nan
    def stars(p):
        if pd.isna(p): return ""
        return "****" if p < 1e-4 else ("***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "")))
    g['emm_stars'] = g['p_adj'].apply(stars)
    return g[['model','condition','delta_emm','delta_emm_lo','delta_emm_hi','SE_delta','p_adj','emm_stars']]

emm_delta = (emm2.groupby('model', group_keys=False).apply(holm_stars)
             .reset_index(drop=True))

# add baseline rows (Δ=0)
baseline_rows = (emm2[emm2['condition']=='baseline'][['model','condition']].copy())
baseline_rows['delta_emm'] = 0.0
baseline_rows['delta_emm_lo'] = 0.0
baseline_rows['delta_emm_hi'] = 0.0
baseline_rows['SE_delta'] = np.nan
baseline_rows['p_adj'] = np.nan
baseline_rows['emm_stars'] = ""
emm_delta = pd.concat([emm_delta, baseline_rows], ignore_index=True)

# -----------------------------
# 3) ΔHedges/100 vs baseline within model
# -----------------------------
tbl = summary_df.copy()
base_h = (tbl[tbl['condition']=='baseline'][['model','hedges_per_100']]
          .rename(columns={'hedges_per_100':'base_h'}))
tbl = tbl.merge(base_h, on='model', how='left')
tbl['delta_hedges'] = tbl['hedges_per_100'] - tbl['base_h']

# (Optional) hedges_sig lookup if you computed Holm tests for hedges:
hedges_sig = {}
try:
    tmp = hedge_results_df.copy()
    tmp['condition'] = tmp['condition'].replace(fix)
    def stars(p): 
        if pd.isna(p): return ""
        return "****" if p < 1e-4 else ("***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "")))
    tmp['hedge_stars'] = tmp['p_adj'].apply(stars)
    hedges_sig = {(r.model, r.condition): r.hedge_stars for r in tmp.itertuples(index=False)}
except NameError:
    pass  # ok if you haven't run hedging tests

# -----------------------------
# 4) Merge Δ–Δ
# -----------------------------
dd = (tbl[['model','condition','delta_hedges']]
      .merge(emm_delta, on=['model','condition'], how='inner'))

dd['model'] = pd.Categorical(dd['model'], categories=model_order, ordered=True)
dd['condition'] = pd.Categorical(dd['condition'], categories=cond_order, ordered=True)
dd = dd.sort_values(['model','condition'])

# -----------------------------
# 5) Plot Δ–Δ (annotate ALL points)
# -----------------------------
fig, axes = plt.subplots(2, 2, figsize=(12, 9), sharex=True, sharey=True)
axes = axes.ravel()

xpad = 0.05 * np.nanmax(np.abs(dd['delta_emm'].values)) if np.isfinite(np.nanmax(np.abs(dd['delta_emm'].values))) else 0.2
ypad = 0.05 * np.nanmax(np.abs(dd['delta_hedges'].values)) if np.isfinite(np.nanmax(np.abs(dd['delta_hedges'].values))) else 0.2
xmax = np.nanmax(np.abs(dd['delta_emm'].values)) + xpad
ymax = np.nanmax(np.abs(dd['delta_hedges'].values)) + ypad

PT_SIZE = 48  # constant point size

for i, m in enumerate(model_order):
    ax = axes[i]
    d = dd[dd['model']==m].copy()
    d = d[d['condition']!='baseline']  # drop baseline point to reduce clutter

    # zero lines
    ax.axvline(0, linestyle='--', linewidth=0.8, alpha=0.6)
    ax.axhline(0, linestyle='--', linewidth=0.8, alpha=0.6)

    # points (constant size)
    ax.scatter(d['delta_emm'], d['delta_hedges'], s=PT_SIZE)

    # ΔEMM error bars only where significant (declutter)
    d_err = d[d['emm_stars'].astype(str).str.len().gt(0)]
    ax.errorbar(d_err['delta_emm'], d_err['delta_hedges'],
                xerr=[d_err['delta_emm'] - d_err['delta_emm_lo'],
                      d_err['delta_emm_hi'] - d_err['delta_emm']],
                fmt='none', linewidth=1, alpha=0.9, capsize=2)

    # stars for ΔEMM significance (above the point)
    yoff = 0.03 * (ymax if np.isfinite(ymax) else 1.0)
    for x, y, star in zip(d['delta_emm'], d['delta_hedges'], d['emm_stars']):
        if isinstance(star, str) and star:
            ax.text(x, y + yoff, star, ha='center', va='bottom', fontsize=11, weight='bold')

    # LABEL **ALL** points with persona abbreviations
    annotate_points(ax, d, xcol='delta_emm', ycol='delta_hedges', label_col='condition')

    ax.set_title(m, weight='bold')
    ax.set_xlim(-xmax, xmax)
    ax.set_ylim(-ymax, ymax)
    ax.grid(True, alpha=0.2)

fig.suptitle('LLM skin tone assessments versus hedging terms used in LLM Rationale', y=0.98, weight='bold')
for ax in axes[::2]: ax.set_ylabel('Δ Hedges per 100 words')
for ax in axes[2:]:  ax.set_xlabel('Difference in skin tone rating (persona - baseline))')

plt.tight_layout(rect=[0,0,1,0.94])
plt.savefig(os.path.join(".", "figure_deltaEMM_vs_deltaHedges_by_model.png"), dpi=600, bbox_inches='tight')
plt.savefig(os.path.join(".", "figure_deltaEMM_vs_deltaHedges_by_model.pdf"), bbox_inches='tight')
plt.show()


In [ ]:
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
from scipy import stats
import matplotlib.patheffects as pe

# -----------------------------
# Config: show long names above stars?
# -----------------------------
SHOW_LONG_ABOVE = True   # set to False to avoid duplicate long names

# load summary df 
summary_df = pd.read_csv(os.path.join(output_path, 'rationale_summary_by_model_condition.csv'))

# FULL, human-readable names (used for significant labels)
pretty_map = {
    'baseline':'Baseline',
    'ignore_world':'Ignore world knowledge',
    'embrace_world':'Embrace world knowledge',
    'cultural_white':'White',
    'cultural_black':'Black',
    'cultural_asian':'Asian',
    'cultural_hispanic':'Hispanic',
    'cultural_indigenous':'Indigenous',
    'cultural_africanamerican':'African American',
    'cultural_colorism':'Colorism',
    'cultural_lighter':'Lighter Community',
    'cultural_darker':'Darker Community'
}

# Abbreviations (used for non-significant labels under the dot)
SHORT = {
    'baseline':'BL', 'ignore_world':'IW', 'embrace_world':'EW',
    'cultural_white':'WH', 'cultural_black':'BK', 'cultural_asian':'AS',
    'cultural_hispanic':'HI', 'cultural_indigenous':'IN',
    'cultural_africanamerican':'AA', 'cultural_colorism':'CL',
    'cultural_lighter':'LG', 'cultural_darker':'DK'
}

# ---------------- helpers ----------------
def label_sig_points_above_stars(ax, d, yoff, pad, xcol='delta_emm', ycol='delta_hedges',
                                 label_col='condition', star_col='emm_stars'):
    """Place FULL persona labels directly ABOVE the star for significant points."""
    used = []
    sig = d[d[star_col].astype(str).str.len().gt(0)].copy()
    for _, row in sig.iterrows():
        x, y = row[xcol], row[ycol]
        tx, ty = x, y + yoff + pad  # label directly above the star
        # simple de-overlap: nudge upward if colliding
        for _ in range(10):
            if any(abs(tx-ux) < 0.03 and abs(ty-uy) < 0.03 for (ux, uy) in used):
                ty += pad * 0.6
            else:
                break
        used.append((tx, ty))
        lab = pretty_map.get(str(row[label_col]), str(row[label_col]))
        ax.text(tx, ty, lab, ha='center', va='bottom', fontsize=9, weight='bold',
                path_effects=[pe.withStroke(linewidth=3, foreground='white')])

def label_below_mixed(ax, d, pad, xcol='delta_emm', ycol='delta_hedges',
                      label_col='condition', star_col='emm_stars'):
    """
    Put labels UNDER every dot:
      - if significant (has star), use FULL persona name,
      - else use abbreviation (SHORT).
    """
    for _, row in d.iterrows():
        x, y = row[xcol], row[ycol]
        is_sig = isinstance(row.get(star_col, ""), str) and len(str(row.get(star_col, ""))) > 0
        lab = pretty_map.get(str(row[label_col]), str(row[label_col])) if is_sig \
              else SHORT.get(str(row[label_col]), str(row[label_col]))
        ty = y - pad
        ax.text(x, ty, lab, ha='center', va='top', fontsize=9, weight='bold',
                path_effects=[pe.withStroke(linewidth=3, foreground='white')])

# -----------------------------
# 0) Inputs
# -----------------------------
emm_path = "outputs/cross_model_emms.csv"
emm = pd.read_csv(emm_path)

# -----------------------------
# 1) Standardize condition keys
# -----------------------------
fix = {'cultural_asain': 'cultural_asian'}
for d in (emm, summary_df):
    d['condition'] = d['condition'].replace(fix)

model_order = ['Claude','GPT','Gemini','Llama']
cond_order = ['baseline','cultural_africanamerican','cultural_asian','cultural_black',
              'cultural_colorism','cultural_darker','cultural_hispanic','cultural_indigenous',
              'cultural_lighter','cultural_white','embrace_world','ignore_world']

# -----------------------------
# 2) ΔEMM vs baseline within model
# -----------------------------
base_emm = (emm[emm['condition']=='baseline']
            .set_index('model')[['emmean','SE']]
            .rename(columns={'emmean':'base_emm','SE':'base_SE'}))

emm2 = emm.merge(base_emm, left_on='model', right_index=True, how='left')
emm2['delta_emm'] = emm2['emmean'] - emm2['base_emm']
emm2['SE_delta']  = np.sqrt(emm2['SE']**2 + emm2['base_SE']**2)  # conservative approx

zcrit = stats.norm.ppf(0.975)
emm2['delta_emm_lo'] = emm2['delta_emm'] - zcrit*emm2['SE_delta']
emm2['delta_emm_hi'] = emm2['delta_emm'] + zcrit*emm2['SE_delta']

def holm_stars(group):
    g = group[group['condition']!='baseline'].copy()
    g['z'] = g['delta_emm'] / g['SE_delta'].replace(0, np.nan)
    g['p_raw'] = 2*(1-stats.norm.cdf(np.abs(g['z'])))
    if g['p_raw'].notna().any():
        mask = g['p_raw'].notna().values
        p_adj = np.full(len(g), np.nan)
        p_adj[mask] = multipletests(g.loc[mask, 'p_raw'], method='holm')[1]
        g['p_adj'] = p_adj
    else:
        g['p_adj'] = np.nan
    def stars(p):
        if pd.isna(p): return ""
        return "****" if p < 1e-4 else ("***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "")))
    g['emm_stars'] = g['p_adj'].apply(stars)
    return g[['model','condition','delta_emm','delta_emm_lo','delta_emm_hi','SE_delta','p_adj','emm_stars']]

emm_delta = (emm2.groupby('model', group_keys=False).apply(holm_stars)
             .reset_index(drop=True))

# add baseline rows (Δ=0)
baseline_rows = (emm2[emm2['condition']=='baseline'][['model','condition']].copy())
baseline_rows['delta_emm'] = 0.0
baseline_rows['delta_emm_lo'] = 0.0
baseline_rows['delta_emm_hi'] = 0.0
baseline_rows['SE_delta'] = np.nan
baseline_rows['p_adj'] = np.nan
baseline_rows['emm_stars'] = ""
emm_delta = pd.concat([emm_delta, baseline_rows], ignore_index=True)

# -----------------------------
# 3) ΔHedges/100 vs baseline within model
# -----------------------------
tbl = summary_df.copy()
base_h = (tbl[tbl['condition']=='baseline'][['model','hedges_per_100']]
          .rename(columns={'hedges_per_100':'base_h'}))
tbl = tbl.merge(base_h, on='model', how='left')
tbl['delta_hedges'] = tbl['hedges_per_100'] - tbl['base_h']

# -----------------------------
# 4) Merge Δ–Δ
# -----------------------------
dd = (tbl[['model','condition','delta_hedges']]
      .merge(emm_delta, on=['model','condition'], how='inner'))

dd['model'] = pd.Categorical(dd['model'], categories=model_order, ordered=True)
dd['condition'] = pd.Categorical(dd['condition'], categories=cond_order, ordered=True)
dd = dd.sort_values(['model','condition'])

# -----------------------------
# 5) Plot Δ–Δ
# -----------------------------
fig, axes = plt.subplots(2, 2, figsize=(12, 9), sharex=True, sharey=True)
axes = axes.ravel()

xpad = 0.05 * np.nanmax(np.abs(dd['delta_emm'].values)) if np.isfinite(np.nanmax(np.abs(dd['delta_emm'].values))) else 0.2
ypad = 0.05 * np.nanmax(np.abs(dd['delta_hedges'].values)) if np.isfinite(np.nanmax(np.abs(dd['delta_hedges'].values))) else 0.2
xmax = np.nanmax(np.abs(dd['delta_emm'].values)) + xpad
ymax = np.nanmax(np.abs(dd['delta_hedges'].values)) + ypad

PT_SIZE = 48  # constant point size

for i, m in enumerate(model_order):
    ax = axes[i]
    d = dd[dd['model']==m].copy()
    d = d[d['condition']!='baseline']  # drop baseline point to reduce clutter

    # zero lines
    ax.axvline(0, linestyle='--', linewidth=0.8, alpha=0.6)
    ax.axhline(0, linestyle='--', linewidth=0.8, alpha=0.6)

    # points (constant size)
    ax.scatter(d['delta_emm'], d['delta_hedges'], s=PT_SIZE)

    # ΔEMM error bars only where significant (declutter)
    d_err = d[d['emm_stars'].astype(str).str.len().gt(0)]
    ax.errorbar(d_err['delta_emm'], d_err['delta_hedges'],
                xerr=[d_err['delta_emm'] - d_err['delta_emm_lo'],
                      d_err['delta_emm_hi'] - d_err['delta_emm']],
                fmt='none', linewidth=1, alpha=0.9, capsize=2)

    # stars for ΔEMM significance (just above each significant point)
    yoff = 0.03 * (ymax if np.isfinite(ymax) else 1.0)
    for x, y, star in zip(d['delta_emm'], d['delta_hedges'], d['emm_stars']):
        if isinstance(star, str) and star:
            ax.text(x, y + yoff, star, ha='center', va='bottom', fontsize=11, weight='bold')

    # NEW: under-dot labels: FULL name if significant, else abbreviation
    label_pad_below = 0.05 * (ymax if np.isfinite(ymax) else 1.0)
    label_below_mixed(ax, d, pad=label_pad_below,
                      xcol='delta_emm', ycol='delta_hedges',
                      label_col='condition', star_col='emm_stars')

    ax.set_title(m, weight='bold')
    ax.set_xlim(-xmax, xmax)
    ax.set_ylim(-ymax, ymax)
    ax.grid(True, alpha=0.2)

fig.suptitle('LLM skin tone assessments versus hedging terms used in LLM Rationale', y=0.98, weight='bold')
for ax in axes[::2]: ax.set_ylabel('Δ Hedges per 100 words')
for ax in axes[2:]:  ax.set_xlabel('Difference in skin tone rating (persona − baseline)')

plt.tight_layout(rect=[0,0,1,0.94])
plt.savefig(os.path.join(".", "figure_deltaEMM_vs_deltaHedges_by_model.png"), dpi=600, bbox_inches='tight')
plt.savefig(os.path.join(".", "figure_deltaEMM_vs_deltaHedges_by_model.pdf"), bbox_inches='tight')
plt.show()


In [ ]:
def plot_delta_emm_by_persona(
    emm_delta: pd.DataFrame,
    cond_order: list,
    model_order: list,
    pretty_map: dict,
    show_long_labels: bool = True,
    save_png: str = None,
    save_pdf: str = None
):
    """
    Plot ΔEMM (persona - baseline) on Y, persona on X, faceted by model.
    Requires columns in emm_delta: ['model','condition','delta_emm','delta_emm_lo','delta_emm_hi','emm_stars'].
    """
    # keep persona rows (drop baseline)
    df = emm_delta.copy()
    df = df[df['condition'] != 'baseline'].copy()

    # order categorical axes
    df['model'] = pd.Categorical(df['model'], categories=model_order, ordered=True)
    df['condition'] = pd.Categorical(df['condition'], categories=[c for c in cond_order if c != 'baseline'], ordered=True)
    df = df.sort_values(['model','condition'])

    # build x tick labels
    def _lab(c):
        return pretty_map.get(str(c), str(c)) if show_long_labels else str(c)
    x_labels = [_lab(c) for c in df['condition'].cat.categories]

    # figure/grid
    n_models = len(model_order)
    ncols = 2
    nrows = int(np.ceil(n_models / ncols))
    fig, axes = plt.subplots(nrows, ncols, figsize=(max(10, 0.9*len(x_labels)*ncols), 4.5*nrows), sharey=True)
    if not isinstance(axes, np.ndarray):
        axes = np.array([axes])
    axes = axes.ravel()

    # y-limits padding
    ymax = np.nanmax(np.abs(df['delta_emm'].values))
    if not np.isfinite(ymax):
        ymax = 1.0
    pad = 0.1 * ymax
    y_lim = (-ymax - pad, ymax + pad)

    for i, m in enumerate(model_order):
        ax = axes[i]
        d = df[df['model'] == m].copy()
        # x positions follow the global condition category order
        xs = np.arange(len(df['condition'].cat.categories))
        # align data to that order
        d = d.set_index('condition').reindex(df['condition'].cat.categories).reset_index()

        # zero line
        ax.axhline(0, linestyle='--', linewidth=0.8, alpha=0.6)

        # points
        ax.scatter(xs, d['delta_emm'], s=40, zorder=3)

        # error bars (use precomputed CI)
        xerr_lo = d['delta_emm'] - d['delta_emm_lo']
        xerr_hi = d['delta_emm_hi'] - d['delta_emm']
        # guard NaNs
        eb_mask = d[['delta_emm_lo','delta_emm_hi']].notna().all(axis=1)
        ax.vlines(xs[eb_mask], d.loc[eb_mask, 'delta_emm_lo'], d.loc[eb_mask, 'delta_emm_hi'], linewidth=2, alpha=0.9)

        # stars above points
        yoff = 0.03 * (y_lim[1] - y_lim[0])
        for x, y, star in zip(xs, d['delta_emm'], d['emm_stars'].fillna('')):
            if isinstance(star, str) and len(star) > 0:
                ax.text(x, y + yoff, star, ha='center', va='bottom', fontsize=11, weight='bold')

        ax.set_title(m, weight='bold')
        ax.set_xticks(xs)
        ax.set_xticklabels(x_labels, rotation=35, ha='right')
        ax.set_ylim(*y_lim)
        ax.grid(True, alpha=0.2)

    # hide any unused axes
    for j in range(i+1, len(axes)):
        axes[j].axis('off')

    fig.suptitle('Difference in skin tone rating (persona − baseline) by persona', y=0.98, weight='bold')
    for r in range(nrows):
        axes[r*ncols].set_ylabel('Δ Skin tone rating (persona − baseline)')
    # x-label on bottom row
    for c in range((nrows-1)*ncols, nrows*ncols):
        if c < len(axes) and axes[c].has_data():
            axes[c].set_xlabel('Persona')

    plt.tight_layout(rect=[0, 0, 1, 0.94])

    if save_png:
        plt.savefig(save_png, dpi=600, bbox_inches='tight')
    if save_pdf:
        plt.savefig(save_pdf, bbox_inches='tight')
    plt.show()


plot_delta_emm_by_persona(
    emm_delta=emm_delta,
    cond_order=cond_order,
    model_order=model_order,
    pretty_map=pretty_map,
    show_long_labels=True,
    save_png=os.path.join(".", "figure_deltaEMM_by_persona_by_model.png"),
    save_pdf=os.path.join(".", "figure_deltaEMM_by_persona_by_model.pdf"),
)


In [ ]:
def plot_delta_emm_combined(
    emm_delta: pd.DataFrame,
    cond_order: list,
    model_order: list,
    pretty_map: dict,
    show_long_labels: bool = True,
    save_png: str = None,
    save_pdf: str = None,
    max_offset: float = 0.18
):
    """
    Combined plot of ΔEMM (persona - baseline) on Y, persona on X, with all models on a single axes.
    Expects columns in emm_delta: ['model','condition','delta_emm','delta_emm_lo','delta_emm_hi','emm_stars'].
    Parameters:
        - emm_delta: DataFrame
        - cond_order: list of conditions (including 'baseline')
        - model_order: list of models (order for legend / offsets)
        - pretty_map: mapping for condition display labels
        - show_long_labels: whether to use pretty_map for x labels
        - save_png/save_pdf: optional paths
        - max_offset: horizontal spread for model points around a persona tick
    """
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import matplotlib.lines as mlines
    from itertools import cycle

    # copy & drop baseline rows (we plot delta = persona - baseline)
    df = emm_delta.copy()
    df = df[df['condition'] != 'baseline'].copy()

    # categorical ordering for conditions and models
    df['model'] = pd.Categorical(df['model'], categories=model_order, ordered=True)
    conds_no_base = [c for c in cond_order if c != 'baseline']
    df['condition'] = pd.Categorical(df['condition'], categories=conds_no_base, ordered=True)
    df = df.sort_values(['condition','model'])  # sort by persona then model for stable plotting

    # x labels
    def _lab(c):
        return pretty_map.get(str(c), str(c)) if show_long_labels else str(c)
    x_labels = [_lab(c) for c in df['condition'].cat.categories]
    personas = df['condition'].cat.categories.tolist()

    # color mapping (same families as prior)
    model_colors = {
        'GPT': '#2ecc71',    # Green
        'Claude': '#e74c3c', # Red
        'Llama': '#3498db',  # Blue
        'Gemini': '#f1c40f', # Yellow
    }
    default_palette = ['#8e44ad', '#16a085', '#d35400', '#2c3e50', '#7f8c8d']
    default_cycle = cycle(default_palette)

    models_present = [m for m in model_order if m in df['model'].cat.categories or m in df['model'].unique()]
    if len(models_present) == 0:
        raise ValueError("No models found in dataframe according to model_order.")

    # assign color per model (try to map family by name)
    model_to_color = {}
    for m in models_present:
        family = next((family for family in model_colors if family.lower() in str(m).lower()), None)
        model_to_color[m] = model_colors.get(family, next(default_cycle))

    # x base positions
    x_base = np.arange(len(personas))

    # prepare offsets for models so multiple models at same persona spread slightly
    n_models = len(models_present)
    if n_models > 1:
        offsets = np.linspace(-max_offset, max_offset, n_models)
    else:
        offsets = np.array([0.0])

    # compute symmetric y-limits (based on delta_emm values)
    ymax = np.nanmax(np.abs(df['delta_emm'].values))
    if not np.isfinite(ymax):
        ymax = 1.0
    pad = 0.12 * ymax
    y_lim = (-ymax - pad, ymax + pad)

    # create plot
    fig, ax = plt.subplots(figsize=(max(10, 0.9*len(x_labels)), 6))

    # zero line
    ax.axhline(0, linestyle='--', linewidth=0.9, alpha=0.6)

    # for legend handles (dots only)
    legend_handles = []

    # Plot models one-by-one using offsets
    for i, model in enumerate(models_present):
        offset = offsets[i]
        color = model_to_color[model]
        # subset
        mdf = df[df['model'] == model].set_index('condition').reindex(personas).reset_index()
        means = mdf['delta_emm'].values.astype(float)
        lo = mdf['delta_emm_lo'].values.astype(float)
        hi = mdf['delta_emm_hi'].values.astype(float)

        # x positions aligned to persona ticks with offset
        xs = x_base + offset

        # plot points (skip NaNs)
        valid = ~np.isnan(means)
        if valid.any():
            ax.scatter(xs[valid], means[valid], s=50, color=color, edgecolor='k', linewidth=0.4, zorder=4)

            # vertical error bars (precomputed lower/upper bounds)
            # ensure we only draw for rows with finite lo/hi
            eb_mask = (~np.isnan(lo)) & (~np.isnan(hi))
            if eb_mask.any():
                ax.vlines(xs[eb_mask], lo[eb_mask], hi[eb_mask], linewidth=2, alpha=0.9, color=color, zorder=3)

            # significance stars above points
            yoff = 0.03 * (y_lim[1] - y_lim[0])
            for x_val, y_val, star in zip(xs[valid], means[valid], mdf.loc[valid, 'emm_stars'].fillna('')):
                if isinstance(star, str) and len(star) > 0:
                    ax.text(x_val, y_val + yoff, star, ha='center', va='bottom', fontsize=11, weight='bold', zorder=6)

        # add a legend handle (dot only)
        legend_handles.append(mlines.Line2D([], [], color=color, marker='o', linestyle='None', markersize=7, label=model))

    # finalize axes
    ax.set_xticks(x_base)
    ax.set_xticklabels(x_labels, rotation=35, ha='right', fontsize=10)
    ax.set_ylim(*y_lim)
    ax.set_ylabel('Δ Skin tone rating (persona − baseline)')
    ax.set_xlabel('Persona')
    ax.set_title('Difference in skin tone rating (persona − baseline) — all models', weight='bold')

    ax.grid(axis='y', linestyle='--', alpha=0.25)

    # legend (dots only)
    ax.legend(handles=legend_handles, title='Model', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)

    plt.tight_layout(rect=[0, 0, 0.88, 0.95])

    # save if requested
    if save_png:
        plt.savefig(save_png, dpi=600, bbox_inches='tight')
    if save_pdf:
        plt.savefig(save_pdf, bbox_inches='tight')
    plt.show()

plot_delta_emm_combined(
    emm_delta=emm_delta,
    cond_order=cond_order,
    model_order=model_order,
    pretty_map=pretty_map,
    show_long_labels=True,
    save_png=os.path.join(".", "figure_deltaEMM_by_persona_combined.png"),
    save_pdf=os.path.join(".", "figure_deltaEMM_by_persona_combined.pdf"),
)

In [ ]:
# Compute within-model shift range (max change across personas within each model)
within_model = (
    emm_delta[emm_delta["condition"] != "baseline"]
    .groupby("model")["delta_emm"]
    .agg(lambda x: x.max() - x.min())
)
max_within_model_shift = within_model.max().round(2)

# Compute between-model shift range for the same persona
# (how much the same persona's delta differs across models)
between_model = (
    emm_delta[emm_delta["condition"] != "baseline"]
    .groupby("condition")["delta_emm"]
    .agg(lambda x: x.max() - x.min())
)
max_between_model_shift = between_model.max().round(2)

print(f"Within a model, changes reached {max_within_model_shift} points; "
      f"between models, ratings for the same persona differed by up to {max_between_model_shift} points.")
